# Supplemental Code
- Inherent instability of simple DNA repeats shapes an evolutionarily stable distribution of repeat lengths 
- McGinty et al. 2025
- Part 4 of 4

In [ ]:
import pandas as pd
import numpy as np
import pickle
import gzip
import os
import kaleido    # https://github.com/plotly/Kaleido
import plotly     # https://plotly.com/python/getting-started/
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'iframe'
pio.templates.default = "none"
pd.set_option('display.precision', 3)
go_config = {'showTips': False}

## Load data

In [ ]:
CHM13_counts = pd.read_pickle('repeat_distributions/CHM13_counts.pickle')
CHM13_counts_B = pd.read_pickle('repeat_distributions/CHM13_counts_Bdist.pickle')
random_counts = pd.read_pickle('repeat_distributions/random_counts.pickle')
random_counts_B = pd.read_pickle('repeat_distributions/random_counts_Bdist.pickle')
subonly_counts = pd.read_pickle('repeat_distributions/subonly_counts_remake.pickle')
primate_counts = pd.read_pickle('repeat_distributions/primates_counts_all.pickle')

denovo_exp_rate = pd.read_pickle('denovo/denovo_exp_rate_remake.pickle')
denovo_con_rate = pd.read_pickle('denovo/denovo_con_rate_remake.pickle')
denovo_nonexp_rate = pd.read_pickle('denovo/denovo_nonexp_rate_remake.pickle')
denovo_substitution_context_rate = pd.read_pickle('denovo/denovo_mut_freq_all.pickle')
decode_exp_rate = pd.read_pickle('decode/decode_expansion_rates.pickle')
decode_con_rate = pd.read_pickle('decode/decode_contraction_rates.pickle')

intercept_list = [denovo_con_rate['A'][8]] + [denovo_exp_rate['A'][8] * (denovo_exp_rate['A'][8]/ denovo_con_rate['A'][8])**x for x in range(7)]

repeats_1_3 = ['A', 'C', 'AC', 'AT', 'AG', 'CG', 'AAT', 'AAG', 'AAC', 'ATC', 'ACT', 'AGG', 'AGC', 'ACG', 'ACC', 'CCG']
bootstrap_counts = pd.read_pickle('repeat_distributions/bootstrap_counts_1000.pickle').sort_index()
bootstrap_counts_max = dict(); bootstrap_counts_min = dict(); bootstrap_counts_mean = dict()
for motif in repeats_1_3:
    bootstrap_counts_mean[motif] = bootstrap_counts.xs(motif, level='motif_RC', axis=1).apply(lambda x: x.sort_values().head(975).tail(950).mean(), axis=1).sort_index().replace(0, np.nan).dropna()
    bootstrap_counts_max[motif] = bootstrap_counts.xs(motif, level='motif_RC', axis=1).apply(lambda x: x.sort_values().head(975).tail(950).max(), axis=1).reindex(bootstrap_counts_mean[motif].index)
    bootstrap_counts_min[motif] = bootstrap_counts.xs(motif, level='motif_RC', axis=1).apply(lambda x: x.sort_values().head(975).tail(950).min(), axis=1).reindex(bootstrap_counts_mean[motif].index)

## Load files

### Gather data from each grid point into single file

In [ ]:
def gather_files(folder, run_name, write='out', final_only = False, check_complete = None, B_dist = False, separator = 'tE|tC|iE|iC', col_list = [6,8,10,12]):
    grid_files = os.listdir(folder)
    grid_files_A = sorted([file for file in grid_files if 'Adist' in file])
    filename_start = pd.Series(grid_files_A).str.split(separator)[0][0]
    filename_end = '_' + pd.Series(grid_files_A).str.split(separator)[0][4].split('_')[1]
    parameter_info = pd.Series(grid_files_A).str.split(separator + '|_', expand=True)[col_list].astype(float)
    parameter_info.index = grid_files_A
    parameter_info.columns = ['exp_p', 'con_p', 'exp_i', 'con_i']
    parameter_info = parameter_info[['exp_i', 'con_i', 'exp_p', 'con_p']]
    parameter_info['exp_i'] = parameter_info['exp_i'].astype(int); parameter_info['con_i'] = parameter_info['con_i'].astype(int)
    parameter_info = parameter_info.sort_index().drop_duplicates(keep = 'first') # if any duplicates exist, keep one with lowest speedup
    parameters_inference_cols = pd.Series([(a,b,c,d) for a,b,c,d in zip(parameter_info['exp_i'], parameter_info['con_i'], parameter_info['exp_p'], parameter_info['con_p'])], index = parameter_info.index)
    
    if check_complete is not None:
        # check for missing files
        jobs = pd.read_pickle(check_complete)
        jobs_incomplete = jobs.loc[jobs.isin(parameters_inference_cols) == False].reset_index(drop=True)
        if len(jobs_incomplete) > 0:
            print('missing ' + str(len(jobs_incomplete)) + ' files')
            jobs_incomplete.to_pickle('jobs_incomplete.pickle')
            return jobs_incomplete
        else:
            print('grid is complete')

    if write != False:
        inference_grid = dict()
        inference_grid_Bdist = dict()
        for file in parameter_info.index:
            try:
                if final_only == False:
                    inference_grid[file] = pd.read_pickle(folder + file)
                else:
                    inference_grid[file] = pd.read_pickle(folder + file).T.tail(1).T
            except pickle.UnpicklingError:
                os.remove(folder + file)
                print('deleted: ' + str(file))
            if B_dist == True:
                inference_grid_Bdist[file] = pd.read_pickle(folder + file.replace('Adist', 'Bdist')).T.tail(1).T
        inference_grid = pd.concat(inference_grid, axis=1).sort_index(axis=1)
        cols = parameter_info.reindex(inference_grid.columns.get_level_values(0))
        cols['round'] = inference_grid.columns.get_level_values(1)
        cols = pd.MultiIndex.from_frame(cols)
        inference_grid.columns = cols
        inference_grid.index +=1
        if write == 'pickle':
            inference_grid.to_pickle('simulations/completed_grids/' + run_name + '.pickle')
        if write == 'csv':
            inference_grid.to_csv('simulations/completed_grids/' + run_name + '.csv', sep = '\t')

        if B_dist == True:
            inference_grid_Bdist = pd.concat(inference_grid_Bdist, axis=1).sort_index(axis=1)
            inference_grid_Bdist.columns = cols
            inference_grid_Bdist.index +=1
            if write == 'pickle':
                inference_grid_Bdist.to_pickle('simulations/completed_grids/' + run_name + '_Bdist.pickle')
            if write == 'csv':
                inference_grid_Bdist.to_csv('simulations/completed_grids/' + run_name + '_Bdist.csv', sep = '\t')

        if write == 'out':
            return inference_grid, inference_grid_Bdist

#### For gathering data from cluster
put the above function into a separate script, along with the below cell:

#### Script parameters
- --dir -> location of DRL files to be combined into one convenient file
- --name -> output name of file
- --write -> type of file to write: csv, pickle or False (to check completeness only)
- --final -> only gather the final timepoint
- --check_complete -> takes the job list as an input and checks for completeness, returning a new job list if any files are missing
- --B_dist -> return file with info on the DRL for B strings
- --sep -> provide alternate file name delimiters (if parameters are named differently than default)
- --col_list -> provide location of parameter info within the file name text string

#### Example commands
(change path and name as needed)

### Load computational model data
(change path and name as needed)
- note: these parameterizations were completed based on pre-defined grids
- other computational model runs focus on reanalysis of data analyzed below and are loaded within the relevant section

In [ ]:
# Power-law parameterization
inference_grid_final_nointerp_subonly = pd.read_pickle('simulations/completed_grids/grid_newpro_PL_final.pickle').droplevel(4, axis=1)
inference_grid_final_nointerp_subonly_Bdist = pd.read_pickle('simulations/completed_grids/grid_newpro_PL_final_Bdist.pickle').droplevel(4, axis=1)
inference_grid_nointerp_subonly = pd.read_pickle('simulations/completed_grids/grid_newpro_PL.pickle')

In [ ]:
# Power-law parameterization with interpolated rates
inference_grid_final_interp_subonly = pd.read_pickle('simulations/completed_grids/grid_3M_PL_subonly_interp_final.pickle').droplevel(4, axis=1)#.dropna(how = 'all', axis=1)

In [ ]:
# Power-law parameterization with constant time rescaling
inference_grid_final_sparse_constantspeedup_subonly = pd.read_pickle('simulations/completed_grids/grid_4param_PL_constant_subonly_final.pickle').droplevel(4, axis=1)#.dropna(how = 'all', axis=1)
inference_grid_sparse_constantspeedup_subonly = pd.read_pickle('simulations/completed_grids/grid_4param_PL_constant_subonly.pickle')

In [ ]:
# Power-law parameterization with constant time rescaling and alternate initial state
inference_grid_final_sparse_constantspeedup_uniform = pd.read_pickle('simulations/completed_grids/grid_4param_PL_constant_uniform_final.pickle').droplevel(4, axis=1)#.dropna(how = 'all', axis=1)
inference_grid_sparse_constantspeedup_uniform = pd.read_pickle('simulations/completed_grids/grid_4param_PL_constant_uniform.pickle')

In [ ]:
# Pure power-law parameterization
inference_grid_final_xrange = pd.read_pickle('simulations/completed_grids/grid_4param_xaxis_v2.pickle').droplevel(4, axis=1)

In [ ]:
# Logarithmic power parameterization
inference_grid_final_log = pd.read_pickle('simulations/completed_grids/grid_4param_log_v4_final.pickle').droplevel(4, axis=1)

### Load mammalian DRLs

In [ ]:
primate_counts = pd.read_pickle('repeat_distributions/primates_counts_all.pickle')
primate_counts_A = primate_counts[1]['A'].unstack().T.fillna(0).astype(int)

# KL divergence

### Primate range used for inference

In [ ]:
norm_range = range(4,201); L_range = range(4,201)

# calculate Kullback-Leibler divergence, comparing a given DRL to T2T-CHM13
def calc_KL_divergence_CHM13(dist, L_range = range(4,201)):
    return ((CHM13_counts_A_norm_ps.reindex(L_range)) * np.log((CHM13_counts_A_norm_ps.reindex(L_range)) / (dist.reindex(L_range)))).sum()

#### Normalize counts (with pseudocount)

In [ ]:
pseudocount = 1 # using 0.01 or 100 makes little difference in rankings. using 100 reduces the Bayes factor of the 2 parameter model, while having almost no effect elsewhere
CHM13_counts_A_norm_ps = (CHM13_counts[1]['A'].reindex(norm_range).fillna(0) + pseudocount) / (CHM13_counts[1]['A'].reindex(norm_range).fillna(0) + pseudocount).sum()
primate_counts_A_norm_ps = (primate_counts_A.reindex(norm_range).fillna(0) + pseudocount) / (primate_counts_A.reindex(norm_range).fillna(0) + pseudocount).sum()

In [ ]:
# Calculate KL
KL_primate_grid = primate_counts_A_norm_ps.apply(lambda y: calc_KL_divergence_CHM13(y, L_range=L_range))

In [ ]:
KL_primate_grid.sort_values()

In [ ]:
n_cutoff = 2
KL_primate_variance = (KL_primate_grid[1:].sort_values()[:-n_cutoff].max() / 2)**2
def calc_likelihood_from_primate_metric_KL(metric):
    # Gaussian acceptance probability (note: not a true likelihood)
    return np.e**-((metric**2)/(2*KL_primate_variance))

In [ ]:
# Normalized counts for plotting
primate_counts_A_norm = primate_counts_A.div(primate_counts_A.sum()).reindex(range(1,200)).fillna(0)
primate_counts_A_norm_hum = primate_counts_A_norm.where(primate_counts_A > 30) * CHM13_counts[1]['A'].sum()
primate_CI_high = primate_counts_A_norm_hum.T.reindex(KL_primate_grid[1:].sort_values()[:-n_cutoff].index).T.max(axis=1)
primate_CI_low = primate_counts_A_norm_hum.T.reindex(KL_primate_grid[1:].sort_values()[:-n_cutoff].index).T.min(axis=1)
primate_counts_A_norm = primate_counts_A.reindex(norm_range) / primate_counts_A.reindex(norm_range).sum()

## Calculate posterior distributions and Bayes factors

### Power-law model (with empirical rate data)

In [ ]:
# Normalize with pseudocount
inference_grid_final_nointerp_subonly_norm_ps = (inference_grid_final_nointerp_subonly.reindex(norm_range).fillna(0) + pseudocount) / (inference_grid_final_nointerp_subonly.reindex(norm_range).fillna(0) + pseudocount).sum()
# Calculate KL
KL_inference_grid_final_nointerp = inference_grid_final_nointerp_subonly_norm_ps.apply(lambda y: calc_KL_divergence_CHM13(y, L_range=L_range))
# Calculate acceptance probability
grid_likelihood = calc_likelihood_from_primate_metric_KL(KL_inference_grid_final_nointerp)

#### grid of rates

In [ ]:
def pin_power_law(power, pin_rate, pin_len=9, start_len=1, end_len=200):
    denom = (pin_len**power) / pin_rate
    return pd.Series([i**power for i in range(start_len, end_len+1)], index = list(range(start_len,end_len+1))) / denom

def intercept_then_powerlaw(exp_power, con_power, exp_int, con_int, pin_len = 9, A_bins = 200, boot = None, motif = 'A', interp = False, nonexp_factor = 0.01, intercept_list = intercept_list):
    if boot is None:
        bootname = ''
        denovo_exp_rate_current = denovo_exp_rate[motif].copy()
        denovo_con_rate_current = denovo_con_rate[motif].copy()
        denovo_nonexp_rate_current = denovo_nonexp_rate[motif].copy()
    else:
        bootname = '_boot'+str(boot)
        denovo_exp_rate_current = denovo_exp_rate_poisson[motif][boot].copy()
        denovo_con_rate_current = denovo_con_rate_poisson[motif][boot].copy()
        denovo_nonexp_rate_current = denovo_nonexp_rate_poisson[motif][boot].copy()
    exp = pd.concat([denovo_exp_rate_current.reindex(range(pin_len)), pin_power_law(exp_power, intercept_list[exp_int], start_len=pin_len, end_len=A_bins+3)])
    con = pd.concat([denovo_con_rate_current.reindex(range(pin_len)), pin_power_law(con_power, intercept_list[con_int], start_len=pin_len, end_len=A_bins+3)])
    nonexp = pd.concat([denovo_nonexp_rate_current.reindex(range(pin_len)), pin_power_law(exp_power, intercept_list[exp_int] * nonexp_factor, start_len=pin_len, end_len=A_bins+3)])
    nonexpname = '_nex' + str(nonexp_factor)
    nonexp.loc[1] = 0
    if interp == True:
        interpname = '_interp'
        exp.loc[8:13] = np.nan
        con.loc[8:13] = np.nan
        nonexp.loc[8:13] = np.nan
        exp = exp.interpolate(method = 'quadratic')
        con = con.interpolate(method = 'quadratic')
        nonexp = nonexp.interpolate(method = 'quadratic')
    else:
        interpname = ''
    name = '_interceptPL_tE' + str(exp_power) + '_tC' + str(con_power) +'_iE' + str(exp_int) + '_iC' + str(con_int) + nonexpname + interpname + bootname
    return name, exp, con, nonexp

In [ ]:
rates_all = pd.DataFrame([intercept_then_powerlaw(exp_p, con_p, exp_i, con_i, interp = False) for (exp_i, con_i, exp_p, con_p,) in grid_likelihood.index], index = grid_likelihood.index)

#### Calculate Bayes factors, etc.

In [ ]:
def model_summary(grid_likelihood, modelname, priorname, paramname, prior = None, params = ['exp_i', 'con_i', 'exp_p', 'con_p'], use_intercepts = True, grid_drl = inference_grid_final_nointerp_subonly_norm_ps, rates = rates_all):
    grid_likelihood = grid_likelihood.reset_index().set_index(['exp_i', 'con_i', 'exp_p', 'con_p'])[0]
    if prior is None:
        prior = (1/len(grid_likelihood))
        posterior = grid_likelihood * prior
    else:
        prior = prior.reindex(grid_likelihood.index)['prob']
        prior /= prior.sum()
        posterior = grid_likelihood * prior.values
    bayesfactors[modelname][priorname][paramname] = posterior.sum()
    posterior /= posterior.sum()

    drl_weighted_avg[modelname][priorname][paramname] = (grid_drl.T.reindex(grid_likelihood.index).T * posterior.values).sum(axis=1)    
    grid_drl_95c[modelname][priorname][paramname] = grid_drl.T.loc[posterior.sort_values(ascending=False).where(posterior.sort_values(ascending=False).cumsum() < 0.95).dropna().index].copy().T

    grid_rates_95c[modelname][priorname][paramname] = rates.loc[posterior.sort_values(ascending=False).where(posterior.sort_values(ascending=False).cumsum() < 0.95).dropna().index].copy().T
    rates_weighted_exp[modelname][priorname][paramname] = pd.Series(np.average(pd.concat(list(rates[1].reindex(grid_likelihood.index)), axis=1), axis=1, weights = posterior.values))
    rates_weighted_con[modelname][priorname][paramname] = pd.Series(np.average(pd.concat(list(rates[2].reindex(grid_likelihood.index)), axis=1), axis=1, weights = posterior.values))
    
    posterior = posterior.reset_index()
    posterior['M'] = pd.Series(intercept_list / intercept_list[1]).reindex(posterior['exp_i']).values
    if use_intercepts == True:
        posterior['exp_i'] = pd.Series(intercept_list).reindex(posterior['exp_i']).values
        posterior['con_i'] = pd.Series(intercept_list).reindex(posterior['con_i']).values

    best_params[modelname][priorname][paramname] = posterior.sort_values(by = [0], ascending = False).iloc[0].reindex(params)
    avg_params[modelname][priorname][paramname] = pd.Series(np.average(posterior[params], axis=0, weights = posterior[0]), index = params)
    
    posteriors_all[modelname][priorname][paramname] = posterior
    posterior_95c[modelname][priorname][paramname] = posterior.set_index(params)[0].sort_values(ascending = False).where(posterior.set_index(params)[0].sort_values(ascending = False).cumsum() < 0.95).dropna()

In [ ]:
bayesfactors = dict(); best_params = dict(); avg_params = dict(); drl_weighted_avg = dict(); posteriors_all = dict(); posterior_95c = dict(); grid_drl_95c = dict(); rates_weighted_exp = dict(); rates_weighted_con = dict(); grid_rates_95c = dict(); subset_indices = dict()

In [ ]:
bayesfactors['PL'] = dict(); best_params['PL'] = dict(); avg_params['PL'] = dict(); drl_weighted_avg['PL'] = dict(); posteriors_all['PL'] = dict(); posterior_95c['PL'] = dict(); grid_drl_95c['PL'] = dict(); rates_weighted_exp['PL'] = dict(); rates_weighted_con['PL'] = dict(); grid_rates_95c['PL'] = dict(); subset_indices['PL'] = dict()

In [ ]:
# uniform prior
bayesfactors['PL']['uniform'] = dict(); best_params['PL']['uniform'] = dict(); avg_params['PL']['uniform'] = dict(); drl_weighted_avg['PL']['uniform'] = dict(); posteriors_all['PL']['uniform'] = dict(); posterior_95c['PL']['uniform'] = dict(); grid_drl_95c['PL']['uniform'] = dict(); rates_weighted_exp['PL']['uniform'] = dict(); rates_weighted_con['PL']['uniform'] = dict(); grid_rates_95c['PL']['uniform'] = dict()
subset_indices['PL']['4param'] = grid_likelihood.index
model_summary(grid_likelihood, 'PL', 'uniform', '4param', prior = None, params = ['exp_i', 'con_i', 'exp_p', 'con_p'])

# 2 parameter Bayes factor, model where power law is same, intercept is same
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_p'] == partial_grid['con_p']) & (partial_grid['exp_i'] == partial_grid['con_i'])).dropna()
subset_indices['PL']['2param'] = partial_grid.set_index(['exp_i', 'con_i', 'exp_p', 'con_p']).index
model_summary(partial_grid, 'PL', 'uniform', '2param', prior = None, params = ['exp_i', 'exp_p'])

# Bayes factor, 3 parameters with shared powers
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_p'] == partial_grid['con_p'])).dropna()
subset_indices['PL']['3param_sharedpowers'] = partial_grid.set_index(['exp_i', 'con_i', 'exp_p', 'con_p']).index
model_summary(partial_grid, 'PL', 'uniform', '3param_sharedpowers', prior = None, params = ['exp_i', 'con_i', 'exp_p'])

# Bayes factor, 3 parameters with shared constants
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_i'] == partial_grid['con_i'])).dropna()
subset_indices['PL']['3param_sharedconstants'] = partial_grid.set_index(['exp_i', 'con_i', 'exp_p', 'con_p']).index
model_summary(partial_grid, 'PL', 'uniform', '3param_sharedconstants', prior = None, params = ['exp_i', 'exp_p', 'con_p'])

# Multiplier 3 parameter model
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_i'] == 1+ partial_grid['con_i'])).dropna()
subset_indices['PL']['3param_Mult'] = partial_grid.set_index(['exp_i', 'con_i', 'exp_p', 'con_p']).index
model_summary(partial_grid, 'PL', 'uniform', '3param_Mult', prior = None, params = ['M', 'exp_p', 'con_p'])

In [ ]:
# restrictive prior
prior_rates = pd.read_pickle('simulations/grid_4param_v2/prior_restrictive_a.pickle')
prior_rates = prior_rates.reindex(grid_likelihood.index)
prior_rates /= prior_rates.sum()

# restrictive prior
bayesfactors['PL']['restrictive'] = dict(); best_params['PL']['restrictive'] = dict(); avg_params['PL']['restrictive'] = dict(); drl_weighted_avg['PL']['restrictive'] = dict(); posteriors_all['PL']['restrictive'] = dict(); posterior_95c['PL']['restrictive'] = dict(); grid_drl_95c['PL']['restrictive'] = dict(); rates_weighted_exp['PL']['restrictive'] = dict(); rates_weighted_con['PL']['restrictive'] = dict(); grid_rates_95c['PL']['restrictive'] = dict()

model_summary(grid_likelihood, 'PL', 'restrictive', '4param', prior = prior_rates, params = ['exp_i', 'con_i', 'exp_p', 'con_p'])

# 2 parameter Bayes factor, model where power law is same, intercept is same
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_p'] == partial_grid['con_p']) & (partial_grid['exp_i'] == partial_grid['con_i'])).dropna()
model_summary(partial_grid, 'PL', 'restrictive', '2param', prior = prior_rates, params = ['exp_i', 'exp_p'])

# Bayes factor, 3 parameters with shared powers
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_p'] == partial_grid['con_p'])).dropna()
model_summary(partial_grid, 'PL', 'restrictive', '3param_sharedpowers', prior = prior_rates, params = ['exp_i', 'con_i', 'exp_p'])

# Bayes factor, 3 parameters with shared constants
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_i'] == partial_grid['con_i'])).dropna()
model_summary(partial_grid, 'PL', 'restrictive', '3param_sharedconstants', prior = prior_rates, params = ['exp_i', 'exp_p', 'con_p'])

# Multiplier 3 parameter model
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_i'] == 1+ partial_grid['con_i'])).dropna()
model_summary(partial_grid, 'PL', 'restrictive', '3param_Mult', prior = prior_rates, params = ['M', 'exp_p', 'con_p'])

In [ ]:
# permissive prior
prior_rates = pd.read_pickle('simulations/grid_4param_v2/prior_a.pickle')
prior_rates = prior_rates.reindex(grid_likelihood.index)
prior_rates /= prior_rates.sum()

# permissive prior
bayesfactors['PL']['permissive'] = dict(); best_params['PL']['permissive'] = dict(); avg_params['PL']['permissive'] = dict(); drl_weighted_avg['PL']['permissive'] = dict(); posteriors_all['PL']['permissive'] = dict(); posterior_95c['PL']['permissive'] = dict(); grid_drl_95c['PL']['permissive'] = dict(); rates_weighted_exp['PL']['permissive'] = dict(); rates_weighted_con['PL']['permissive'] = dict(); grid_rates_95c['PL']['permissive'] = dict()

model_summary(grid_likelihood, 'PL', 'permissive', '4param', prior = prior_rates, params = ['exp_i', 'con_i', 'exp_p', 'con_p'])

# 2 parameter Bayes factor, model where power law is same, intercept is same
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_p'] == partial_grid['con_p']) & (partial_grid['exp_i'] == partial_grid['con_i'])).dropna()
model_summary(partial_grid, 'PL', 'permissive', '2param', prior = prior_rates, params = ['exp_i', 'exp_p'])

# Bayes factor, 3 parameters with shared powers
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_p'] == partial_grid['con_p'])).dropna()
model_summary(partial_grid, 'PL', 'permissive', '3param_sharedpowers', prior = prior_rates, params = ['exp_i', 'con_i', 'exp_p'])

# Bayes factor, 3 parameters with shared constants
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_i'] == partial_grid['con_i'])).dropna()
model_summary(partial_grid, 'PL', 'permissive', '3param_sharedconstants', prior = prior_rates, params = ['exp_i', 'exp_p', 'con_p'])

# Multiplier 3 parameter model
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_i'] == 1+ partial_grid['con_i'])).dropna()
model_summary(partial_grid, 'PL', 'permissive', '3param_Mult', prior = prior_rates, params = ['M', 'exp_p', 'con_p'])

In [ ]:
avg_params['PL']['uniform'] = pd.concat(avg_params['PL']['uniform']).unstack()
avg_params['PL']['restrictive'] = pd.concat(avg_params['PL']['restrictive']).unstack()
avg_params['PL']['permissive'] = pd.concat(avg_params['PL']['permissive']).unstack()
avg_params['PL'] = pd.concat(avg_params['PL'])

best_params['PL']['uniform'] = pd.concat(best_params['PL']['uniform']).unstack()
best_params['PL']['restrictive'] = pd.concat(best_params['PL']['restrictive']).unstack()
best_params['PL']['permissive'] = pd.concat(best_params['PL']['permissive']).unstack()
best_params['PL'] = pd.concat(best_params['PL'])

drl_weighted_avg['PL']['uniform'] = pd.concat(drl_weighted_avg['PL']['uniform']).unstack()
drl_weighted_avg['PL']['restrictive'] = pd.concat(drl_weighted_avg['PL']['restrictive']).unstack()
drl_weighted_avg['PL']['permissive'] = pd.concat(drl_weighted_avg['PL']['permissive']).unstack()
drl_weighted_avg['PL'] = pd.concat(drl_weighted_avg['PL'])

rates_weighted_exp['PL']['uniform'] = pd.concat(rates_weighted_exp['PL']['uniform']).unstack().T
rates_weighted_exp['PL']['restrictive'] = pd.concat(rates_weighted_exp['PL']['restrictive']).unstack().T
rates_weighted_exp['PL']['permissive'] = pd.concat(rates_weighted_exp['PL']['permissive']).unstack().T
rates_weighted_exp['PL'] = pd.concat(rates_weighted_exp['PL'])

rates_weighted_con['PL']['uniform'] = pd.concat(rates_weighted_con['PL']['uniform']).unstack().T
rates_weighted_con['PL']['restrictive'] = pd.concat(rates_weighted_con['PL']['restrictive']).unstack().T
rates_weighted_con['PL']['permissive'] = pd.concat(rates_weighted_con['PL']['permissive']).unstack().T
rates_weighted_con['PL'] = pd.concat(rates_weighted_con['PL'])

### Power-law model with interpolation

In [ ]:
# Normalize with pseudocount
inference_grid_final_interp_subonly_norm_ps = (inference_grid_final_interp_subonly.reindex(L_range).fillna(0) + pseudocount) / (inference_grid_final_interp_subonly.reindex(L_range).fillna(0) + pseudocount).sum()
# Calculate KL
KL_inference_grid_final_interp = inference_grid_final_interp_subonly_norm_ps.apply(lambda y: calc_KL_divergence_CHM13(y, L_range=L_range))
# Calculate acceptance probability
grid_likelihood = calc_likelihood_from_primate_metric_KL(KL_inference_grid_final_interp)

In [ ]:
# grid of rates
rates_all_interp = pd.DataFrame([intercept_then_powerlaw(exp_p, con_p, exp_i, con_i, interp = True) for (exp_i, con_i, exp_p, con_p,) in grid_likelihood.index], index = grid_likelihood.index)

#### Calculate Bayes factors, etc.

In [ ]:
bayesfactors['PLinterp'] = dict(); best_params['PLinterp'] = dict(); avg_params['PLinterp'] = dict(); drl_weighted_avg['PLinterp'] = dict(); posteriors_all['PLinterp'] = dict(); posterior_95c['PLinterp'] = dict(); grid_drl_95c['PLinterp'] = dict(); rates_weighted_exp['PLinterp'] = dict(); rates_weighted_con['PLinterp'] = dict(); grid_rates_95c['PLinterp'] = dict(); subset_indices['PLinterp'] = dict()

In [ ]:
# uniform prior
bayesfactors['PLinterp']['uniform'] = dict(); best_params['PLinterp']['uniform'] = dict(); avg_params['PLinterp']['uniform'] = dict(); drl_weighted_avg['PLinterp']['uniform'] = dict(); posteriors_all['PLinterp']['uniform'] = dict(); posterior_95c['PLinterp']['uniform'] = dict(); grid_drl_95c['PLinterp']['uniform'] = dict(); rates_weighted_exp['PLinterp']['uniform'] = dict(); rates_weighted_con['PLinterp']['uniform'] = dict(); grid_rates_95c['PLinterp']['uniform'] = dict()

# Multiplier 3 parameter model
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_i'] == 1+ partial_grid['con_i'])).dropna()
subset_indices['PLinterp']['3param_Mult'] = partial_grid.set_index(['exp_i', 'con_i', 'exp_p', 'con_p']).index
model_summary(partial_grid, 'PLinterp', 'uniform', '3param_Mult', prior = None, params = ['M', 'exp_p', 'con_p'], grid_drl = inference_grid_final_interp_subonly_norm_ps, rates=rates_all_interp)

In [ ]:
# restrictive prior
prior_rates = pd.read_pickle('simulations/grid_4param_v2/prior_restrictive_a.pickle')
prior_rates = prior_rates.reindex(grid_likelihood.index)
prior_rates /= prior_rates.sum()

# restrictive prior
bayesfactors['PLinterp']['restrictive'] = dict(); best_params['PLinterp']['restrictive'] = dict(); avg_params['PLinterp']['restrictive'] = dict(); drl_weighted_avg['PLinterp']['restrictive'] = dict(); posteriors_all['PLinterp']['restrictive'] = dict(); posterior_95c['PLinterp']['restrictive'] = dict(); grid_drl_95c['PLinterp']['restrictive'] = dict(); rates_weighted_exp['PLinterp']['restrictive'] = dict(); rates_weighted_con['PLinterp']['restrictive'] = dict(); grid_rates_95c['PLinterp']['restrictive'] = dict()

# Multiplier 3 parameter model
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_i'] == 1+ partial_grid['con_i'])).dropna()
model_summary(partial_grid, 'PLinterp', 'restrictive', '3param_Mult', prior = prior_rates, params = ['M', 'exp_p', 'con_p'], grid_drl = inference_grid_final_interp_subonly_norm_ps, rates=rates_all_interp)

In [ ]:
# permissive prior
prior_rates = pd.read_pickle('simulations/grid_4param_v2/prior_a.pickle')
prior_rates = prior_rates.reindex(grid_likelihood.index)
prior_rates /= prior_rates.sum()

# permissive prior
bayesfactors['PLinterp']['permissive'] = dict(); best_params['PLinterp']['permissive'] = dict(); avg_params['PLinterp']['permissive'] = dict(); drl_weighted_avg['PLinterp']['permissive'] = dict(); posteriors_all['PLinterp']['permissive'] = dict(); posterior_95c['PLinterp']['permissive'] = dict(); grid_drl_95c['PLinterp']['permissive'] = dict(); rates_weighted_exp['PLinterp']['permissive'] = dict(); rates_weighted_con['PLinterp']['permissive'] = dict(); grid_rates_95c['PLinterp']['permissive'] = dict()

# Multiplier 3 parameter model
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_i'] == 1+ partial_grid['con_i'])).dropna()
model_summary(partial_grid, 'PLinterp', 'permissive', '3param_Mult', prior = prior_rates, params = ['M', 'exp_p', 'con_p'], grid_drl = inference_grid_final_interp_subonly_norm_ps, rates=rates_all_interp)

In [ ]:
avg_params['PLinterp']['uniform'] = pd.concat(avg_params['PLinterp']['uniform']).unstack()
avg_params['PLinterp']['restrictive'] = pd.concat(avg_params['PLinterp']['restrictive']).unstack()
avg_params['PLinterp']['permissive'] = pd.concat(avg_params['PLinterp']['permissive']).unstack()
avg_params['PLinterp'] = pd.concat(avg_params['PLinterp'])

best_params['PLinterp']['uniform'] = pd.concat(best_params['PLinterp']['uniform']).unstack()
best_params['PLinterp']['restrictive'] = pd.concat(best_params['PLinterp']['restrictive']).unstack()
best_params['PLinterp']['permissive'] = pd.concat(best_params['PLinterp']['permissive']).unstack()
best_params['PLinterp'] = pd.concat(best_params['PLinterp'])

drl_weighted_avg['PLinterp']['uniform'] = pd.concat(drl_weighted_avg['PLinterp']['uniform']).unstack()
drl_weighted_avg['PLinterp']['restrictive'] = pd.concat(drl_weighted_avg['PLinterp']['restrictive']).unstack()
drl_weighted_avg['PLinterp']['permissive'] = pd.concat(drl_weighted_avg['PLinterp']['permissive']).unstack()
drl_weighted_avg['PLinterp'] = pd.concat(drl_weighted_avg['PLinterp'])

rates_weighted_exp['PLinterp']['uniform'] = pd.concat(rates_weighted_exp['PLinterp']['uniform']).unstack().T
rates_weighted_exp['PLinterp']['restrictive'] = pd.concat(rates_weighted_exp['PLinterp']['restrictive']).unstack().T
rates_weighted_exp['PLinterp']['permissive'] = pd.concat(rates_weighted_exp['PLinterp']['permissive']).unstack().T
rates_weighted_exp['PLinterp'] = pd.concat(rates_weighted_exp['PLinterp'])

rates_weighted_con['PLinterp']['uniform'] = pd.concat(rates_weighted_con['PLinterp']['uniform']).unstack().T
rates_weighted_con['PLinterp']['restrictive'] = pd.concat(rates_weighted_con['PLinterp']['restrictive']).unstack().T
rates_weighted_con['PLinterp']['permissive'] = pd.concat(rates_weighted_con['PLinterp']['permissive']).unstack().T
rates_weighted_con['PLinterp'] = pd.concat(rates_weighted_con['PLinterp'])

## Plot posterior distributions
- posterior contours (highest density range)

In [ ]:
intercept_revlist = pd.Series(intercept_list).reset_index().set_index([0])['index']

In [ ]:
spacing = [0, 0.682689492137086, 0.954499736103642, 0.997300203936740, 0.999936657516334, 1]
discrete_deep = plotly.colors.make_colorscale(plotly.colors.sample_colorscale('deep', np.linspace(0,1,5)))
discrete_deep_range = [[0.0, 'rgb(253, 253, 204)'],
 [0.249999999, 'rgb(253, 253, 204)'],
 [0.25, 'rgb(122, 205, 163)'],
 [0.4999999999, 'rgb(122, 205, 163)'],
 [0.5, 'rgb(72, 142, 158)'],
 [0.7499999999, 'rgb(72, 142, 158)'],
 [0.75, 'rgb(62, 77, 136)'],
 [0.9999999999, 'rgb(62, 77, 136)'],
 [1.0, 'rgb(39, 26, 44)'], ]

#### 4 parameter power-law model

In [ ]:
current_posterior = posteriors_all['PL']['uniform']['4param'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['exp_i', 'con_i', 'exp_p', 'con_p'])

In [ ]:
# Fig. S10a
fig = make_subplots(rows = 8, cols = 8, shared_xaxes = True, shared_yaxes = True, vertical_spacing = 0.015, horizontal_spacing = 0.008, row_titles = ['<i>c</i><sub>𝜅</sub>=' + '{:.2e}'.format(x) for x in intercept_list[::-1]], column_titles = ['<i>c</i><sub>𝜀</sub>=' + '{:.2e}'.format(x) for x in intercept_list], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')
col_count = 0
for exp_i in intercept_list:
    row_count=0; col_count +=1
    for con_i in intercept_list[::-1]:
        row_count+=1
        fig.add_trace(go.Heatmap(x = posterior_contour.loc[exp_i].loc[con_i].index.get_level_values(0), y = posterior_contour.loc[exp_i].loc[con_i].index.get_level_values(1), z=(posterior_contour.loc[exp_i].loc[con_i]['level_0']+1), coloraxis='coloraxis1'), row = row_count, col = col_count)
# best parameter combination
fig.add_annotation(x=current_posterior.iloc[0]['exp_p'], y=current_posterior.iloc[0]['con_p'], showarrow=True, arrowhead=1, arrowcolor = 'red', col = 1 + intercept_revlist.loc[current_posterior.iloc[0]['exp_i']], row = 8 - intercept_revlist.loc[current_posterior.iloc[0]['con_i']])

fig.update_layout(coloraxis1=dict(colorscale=discrete_deep_range, colorbar = dict(title = 'HDR', len=0.25, tickvals = [1.5,2.5,3.5,4.5], ticktext = ['68%', '95%', '99.7%', '99.99%']), cmin = 1, cmax = 5))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 1000, width = 1200, margin={'t':30,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)
fig.show()

In [ ]:
fig.write_image('plots/posterior_dist_contour_PL_4param_uniform.pdf')

In [ ]:
# Fig. S10b
current_posterior = posteriors_all['PL']['permissive']['4param'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['exp_i', 'con_i', 'exp_p', 'con_p'])

fig = make_subplots(rows = 8, cols = 8, shared_xaxes = True, shared_yaxes = True, vertical_spacing = 0.015, horizontal_spacing = 0.008, row_titles = ['<i>c</i><sub>𝜅</sub>=' + '{:.2e}'.format(x) for x in intercept_list[::-1]], column_titles = ['<i>c</i><sub>𝜀</sub>=' + '{:.2e}'.format(x) for x in intercept_list], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')
col_count = 0
for exp_i in intercept_list:
    row_count=0; col_count +=1
    for con_i in intercept_list[::-1]:
        row_count+=1
        fig.add_trace(go.Heatmap(x = posterior_contour.loc[exp_i].loc[con_i].index.get_level_values(0), y = posterior_contour.loc[exp_i].loc[con_i].index.get_level_values(1), z=(posterior_contour.loc[exp_i].loc[con_i]['level_0']+1), coloraxis='coloraxis1'), row = row_count, col = col_count)
# best parameter combination
fig.add_annotation(x=current_posterior.iloc[0]['exp_p'], y=current_posterior.iloc[0]['con_p'], showarrow=True, arrowhead=1, arrowcolor = 'red', col = 1 + intercept_revlist.loc[current_posterior.iloc[0]['exp_i']], row = 8 - intercept_revlist.loc[current_posterior.iloc[0]['con_i']])

fig.update_layout(coloraxis1=dict(colorscale=discrete_deep_range, colorbar = dict(title = 'HDR', len=0.25, tickvals = [1.5,2.5,3.5,4.5], ticktext = ['68%', '95%', '99.7%', '99.99%']), cmin = 1, cmax = 5))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 1000, width = 1200, margin={'t':30,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)
fig.write_image('plots/posterior_dist_contour_PL_4param_permissive.pdf')

In [ ]:
# Fig. S10c
current_posterior = posteriors_all['PL']['restrictive']['4param'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['exp_i', 'con_i', 'exp_p', 'con_p'])

fig = make_subplots(rows = 8, cols = 8, shared_xaxes = True, shared_yaxes = True, vertical_spacing = 0.015, horizontal_spacing = 0.008, row_titles = ['<i>c</i><sub>𝜅</sub>=' + '{:.2e}'.format(x) for x in intercept_list[::-1]], column_titles = ['<i>c</i><sub>𝜀</sub>=' + '{:.2e}'.format(x) for x in intercept_list], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')
col_count = 0
for exp_i in intercept_list:
    row_count=0; col_count +=1
    for con_i in intercept_list[::-1]:
        row_count+=1
        fig.add_trace(go.Heatmap(x = posterior_contour.loc[exp_i].loc[con_i].index.get_level_values(0), y = posterior_contour.loc[exp_i].loc[con_i].index.get_level_values(1), z=(posterior_contour.loc[exp_i].loc[con_i]['level_0']+1), coloraxis='coloraxis1'), row = row_count, col = col_count)
# best parameter combination
fig.add_annotation(x=current_posterior.iloc[0]['exp_p'], y=current_posterior.iloc[0]['con_p'], showarrow=True, arrowhead=1, arrowcolor = 'red', col = 1 + intercept_revlist.loc[current_posterior.iloc[0]['exp_i']], row = 8 - intercept_revlist.loc[current_posterior.iloc[0]['con_i']])

fig.update_layout(coloraxis1=dict(colorscale=discrete_deep_range, colorbar = dict(title = 'HDR', len=0.25, tickvals = [1.5,2.5,3.5,4.5], ticktext = ['68%', '95%', '99.7%', '99.99%']), cmin = 1, cmax = 5))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 1000, width = 1200, margin={'t':30,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)
fig.write_image('plots/posterior_dist_contour_PL_4param_restrictive.pdf')

#### nested model figure (Fig. S16)

In [ ]:
nested = posteriors_all['PL']['uniform']['4param'].copy()
nested['category'] = 1
nested['category'] = 3*nested.where(nested['exp_i'] == nested['con_i'])['category'].fillna(0) + nested.where(nested['exp_p'] == nested['con_p'])['category'].fillna(0) + 2* nested.where(pd.Series(intercept_revlist.reindex(nested['exp_i']).values == intercept_revlist.reindex(nested['con_i']).values +1, index = nested.index))['category'].fillna(0) - 2* nested.where((nested['exp_p'] == nested['con_p']) & pd.Series(intercept_revlist.reindex(nested['exp_i']).values == intercept_revlist.reindex(nested['con_i']).values +1, index = nested.index))['category'].fillna(0)
nested.set_index(['exp_i', 'con_i', 'exp_p', 'con_p'], inplace = True)

In [ ]:
cat_colors = [[0, plotly.colors.DEFAULT_PLOTLY_COLORS[7]],
 [0.19999999, plotly.colors.DEFAULT_PLOTLY_COLORS[7]],
 [0.2, plotly.colors.DEFAULT_PLOTLY_COLORS[1]],
 [0.399999999, plotly.colors.DEFAULT_PLOTLY_COLORS[1]],
 [0.4, plotly.colors.DEFAULT_PLOTLY_COLORS[2]],
 [0.5999999999, plotly.colors.DEFAULT_PLOTLY_COLORS[2]],
 [0.6, plotly.colors.DEFAULT_PLOTLY_COLORS[4]],
 [0.7999999999, plotly.colors.DEFAULT_PLOTLY_COLORS[4]],
 [0.8, plotly.colors.DEFAULT_PLOTLY_COLORS[0]],
 [1, plotly.colors.DEFAULT_PLOTLY_COLORS[0]],]

In [ ]:
fig = make_subplots(rows = 8, cols = 8, shared_xaxes = True, shared_yaxes = True, vertical_spacing = 0.015, horizontal_spacing = 0.008, row_titles = ['<i>c</i><sub>𝜅</sub>=' + '{:.2e}'.format(x) for x in intercept_list[::-1]], column_titles = ['<i>c</i><sub>𝜀</sub>=' + '{:.2e}'.format(x) for x in intercept_list], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')
col_count = 0
for exp_i in intercept_list:
    row_count=0; col_count +=1
    for con_i in intercept_list[::-1]:
        row_count+=1
        fig.add_trace(go.Heatmap(x = nested.loc[exp_i].loc[con_i].index.get_level_values(0), y = nested.loc[exp_i].loc[con_i].index.get_level_values(1), z=(nested.loc[exp_i].loc[con_i]['category']), coloraxis='coloraxis1'), row = row_count, col = col_count)
fig.update_layout(coloraxis1=dict(colorscale=cat_colors, colorbar = dict(title = 'Nested parameterizations:', len=0.25, x = 0.05, y = -0.16, tickvals = [0.4,1.2,2,2.8,3.6], ticktext = ['Decoupled power laws', 'Power laws with independent constants ', 'Multiplier-coupled power laws ', 'Power laws with independent exponents', 'Symmetric power law'])))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 1200, width = 1000, margin={'t':30,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)
fig.show()

In [ ]:
fig.write_image('plots/fig_nested_model_diagram.pdf')

#### 3 parameter multiplier-coupled model

In [ ]:
m_list = intercept_list[1:]/ intercept_list[1]

In [ ]:
# Fig. 5a
fig = make_subplots(rows = 2, cols = 7, shared_xaxes = True, shared_yaxes = True, horizontal_spacing = 0.02, vertical_spacing = 0.08, subplot_titles = ['<i>m</i>=' + str(round(x / intercept_list[1], 1)) for x in intercept_list[1:]], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')

current_posterior = posteriors_all['PL']['uniform']['3param_Mult'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
posterior_contour_3M_pro_uni = posterior_contour.copy()
row_count=1; col_count = 0
for m in m_list:
    col_count+=1
    fig.add_trace(go.Heatmap(x = posterior_contour.loc[m].index.get_level_values(0), y = posterior_contour.loc[m].index.get_level_values(1), z=(posterior_contour.loc[m]['level_0']+1), coloraxis='coloraxis1'), row = row_count, col = col_count)
# best parameter combination
fig.add_annotation(x=current_posterior.iloc[0]['exp_p'], y=current_posterior.iloc[0]['con_p'], showarrow=True, arrowhead=1, arrowcolor = 'red', row = 1, col = pd.Series(range(1,8), index = m_list).loc[current_posterior.iloc[0]['M']])

current_posterior = posteriors_all['PL']['restrictive']['3param_Mult'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
row_count=2; col_count = 0
for m in m_list:
    col_count+=1
    fig.add_trace(go.Heatmap(x = posterior_contour.loc[m].index.get_level_values(0), y = posterior_contour.loc[m].index.get_level_values(1), z=(posterior_contour.loc[m]['level_0']+1), coloraxis='coloraxis1'), row = row_count, col = col_count)
# best parameter combination
fig.add_annotation(x=current_posterior.iloc[0]['exp_p'], y=current_posterior.iloc[0]['con_p'], showarrow=True, arrowhead=1, arrowcolor = 'red', row = 2, col = pd.Series(range(1,8), index = m_list).loc[current_posterior.iloc[0]['M']])

fig.update_layout(coloraxis1=dict(colorscale=discrete_deep_range, colorbar = dict(title = 'HDR', len=0.85, tickvals = [1.5,2.5,3.5,4.5], ticktext = ['68%', '95%', '99.7%', '99.99%'], x = 1.02), cmin = 1, cmax = 5))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 310, width = 1000, margin={'t':50,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)
fig.show()

In [ ]:
fig.write_image('plots/posterior_dist_contour_PL_3mult_uniform_restrictive.pdf')

In [ ]:
# Fig. S7a
fig = make_subplots(rows = 3, cols = 7, shared_xaxes = True, shared_yaxes = True, horizontal_spacing = 0.02, vertical_spacing = 0.08, subplot_titles = ['<i>m</i>=' + str(round(x / intercept_list[1], 1)) for x in intercept_list[1:]], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')

current_posterior = posteriors_all['PL']['uniform']['3param_Mult'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
row_count=1; col_count = 0
for m in m_list:
    col_count+=1
    fig.add_trace(go.Heatmap(x = posterior_contour.loc[m].index.get_level_values(0), y = posterior_contour.loc[m].index.get_level_values(1), z=(posterior_contour.loc[m]['level_0']+1), coloraxis='coloraxis1'), row = row_count, col = col_count)
# best parameter combination
fig.add_annotation(x=current_posterior.iloc[0]['exp_p'], y=current_posterior.iloc[0]['con_p'], showarrow=True, arrowhead=1, arrowcolor = 'red', row = 1, col = pd.Series(range(1,8), index = m_list).loc[current_posterior.iloc[0]['M']])

current_posterior = posteriors_all['PL']['permissive']['3param_Mult'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
row_count=2; col_count = 0
for m in m_list:
    col_count+=1
    fig.add_trace(go.Heatmap(x = posterior_contour.loc[m].index.get_level_values(0), y = posterior_contour.loc[m].index.get_level_values(1), z=(posterior_contour.loc[m]['level_0']+1), coloraxis='coloraxis1'), row = row_count, col = col_count)
# best parameter combination
fig.add_annotation(x=current_posterior.iloc[0]['exp_p'], y=current_posterior.iloc[0]['con_p'], showarrow=True, arrowhead=1, arrowcolor = 'red', row = 2, col = pd.Series(range(1,8), index = m_list).loc[current_posterior.iloc[0]['M']])

current_posterior = posteriors_all['PL']['restrictive']['3param_Mult'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
row_count=3; col_count = 0
for m in m_list:
    col_count+=1
    fig.add_trace(go.Heatmap(x = posterior_contour.loc[m].index.get_level_values(0), y = posterior_contour.loc[m].index.get_level_values(1), z=(posterior_contour.loc[m]['level_0']+1), coloraxis='coloraxis1'), row = row_count, col = col_count)
# best parameter combination
fig.add_annotation(x=current_posterior.iloc[0]['exp_p'], y=current_posterior.iloc[0]['con_p'], showarrow=True, arrowhead=1, arrowcolor = 'red', row = 3, col = pd.Series(range(1,8), index = m_list).loc[current_posterior.iloc[0]['M']])

fig.update_layout(coloraxis1=dict(colorscale=discrete_deep_range, colorbar = dict(title = 'HDR', len=0.55, tickvals = [1.5,2.5,3.5,4.5], ticktext = ['68%', '95%', '99.7%', '99.99%'], x = 1.05), cmin = 1, cmax = 5))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 420, width = 1000, margin={'t':50,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)
fig.show()

In [ ]:
fig.write_image('plots/posterior_dist_contour_PL_3mult_uniform_permissive_restrictive.pdf')

#### 3 parameter power-law model with interpolation (Fig. S8a)

In [ ]:
fig = make_subplots(rows = 3, cols = 7, shared_xaxes = True, shared_yaxes = True, horizontal_spacing = 0.02, vertical_spacing = 0.08, subplot_titles = ['<i>m</i>=' + str(round(x / intercept_list[1], 1)) for x in intercept_list[1:]], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')

current_posterior = posteriors_all['PLinterp']['uniform']['3param_Mult'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
row_count=1; col_count = 0
for m in m_list:
    col_count+=1
    fig.add_trace(go.Heatmap(x = posterior_contour.loc[m].index.get_level_values(0), y = posterior_contour.loc[m].index.get_level_values(1), z=(posterior_contour.loc[m]['level_0']+1), coloraxis='coloraxis1'), row = row_count, col = col_count)
# best parameter combination
fig.add_annotation(x=current_posterior.iloc[0]['exp_p'], y=current_posterior.iloc[0]['con_p'], showarrow=True, arrowhead=1, arrowcolor = 'red', row = 1, col = pd.Series(range(1,8), index = m_list).loc[current_posterior.iloc[0]['M']])

current_posterior = posteriors_all['PLinterp']['permissive']['3param_Mult'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
row_count=2; col_count = 0
for m in m_list:
    col_count+=1
    fig.add_trace(go.Heatmap(x = posterior_contour.loc[m].index.get_level_values(0), y = posterior_contour.loc[m].index.get_level_values(1), z=(posterior_contour.loc[m]['level_0']+1), coloraxis='coloraxis1'), row = row_count, col = col_count)
# best parameter combination
fig.add_annotation(x=current_posterior.iloc[0]['exp_p'], y=current_posterior.iloc[0]['con_p'], showarrow=True, arrowhead=1, arrowcolor = 'red', row = 2, col = pd.Series(range(1,8), index = m_list).loc[current_posterior.iloc[0]['M']])

current_posterior = posteriors_all['PLinterp']['restrictive']['3param_Mult'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
row_count=3; col_count = 0
for m in m_list:
    col_count+=1
    fig.add_trace(go.Heatmap(x = posterior_contour.loc[m].index.get_level_values(0), y = posterior_contour.loc[m].index.get_level_values(1), z=(posterior_contour.loc[m]['level_0']+1), coloraxis='coloraxis1'), row = row_count, col = col_count)
# best parameter combination
fig.add_annotation(x=current_posterior.iloc[0]['exp_p'], y=current_posterior.iloc[0]['con_p'], showarrow=True, arrowhead=1, arrowcolor = 'red', row = 3, col = pd.Series(range(1,8), index = m_list).loc[current_posterior.iloc[0]['M']])

fig.update_layout(coloraxis1=dict(colorscale=discrete_deep_range, colorbar = dict(title = 'HDR', len=0.55, tickvals = [1.5,2.5,3.5,4.5], ticktext = ['68%', '95%', '99.7%', '99.99%'], x = 1.05), cmin = 1, cmax = 5))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 420, width = 1000, margin={'t':50,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)
fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)
fig.show()

In [ ]:
fig.write_image('plots/posterior_dist_contour_PL_3mult_uniform_permissive_restrictive_interp.pdf')

## Plot HDR range and posterior-weighted DRLs and rates

#### Plot functions

In [ ]:
def make_fig_list_contour(grid_drl, contour, motif='A', showgroups = [2,1,0], norm_range = range(4,201), xrange = False, tracename = None):
    fig = go.Figure()

    if norm_range != False:
        grid_drl = grid_drl.copy().mul(bootstrap_counts_mean[motif].reindex(norm_range).sum())

    if contour is not None:
        for group in showgroups:
            current_contour = contour.loc[contour['level_0'] == group].copy().reset_index()
            if xrange == False:
                current_contour['exp_i'] = intercept_revlist.reindex(current_contour['exp_i']).values
                current_contour['con_i'] = intercept_revlist.reindex(current_contour['con_i']).values
            current_contour.set_index(['exp_i', 'con_i', 'exp_p', 'con_p'], inplace = True)
            current_drl = grid_drl.reindex(current_contour.index, axis=1)
            
            fig.add_trace(go.Scatter(x = (current_drl.index) * len(motif), y = current_drl.min(axis=1), mode = 'lines', line = dict(width=0 if list(current_drl.min(axis=1)) != list(current_drl.max(axis=1)) else 4, color = discrete_deep[group][1]), legendgroup = group, showlegend = False, name = group if tracename is None else tracename))
            fig.add_trace(go.Scatter(x = (current_drl.index) * len(motif), y = current_drl.max(axis=1), fill = 'tonexty', fillcolor = 'rgba(' + pd.DataFrame(discrete_deep)[1][group][4:-1] + ', 0.8)' if len(showgroups) != 1 else 'rgba(' + pd.DataFrame(discrete_deep)[1][1][4:-1] + ', 0.5)', mode = 'lines', line = dict(width = 0, color = discrete_deep[group][1]), legendgroup = group, name = group if tracename is None else tracename))
    

    fig.add_trace(go.Scatter(x = primate_CI_high.dropna().index, y= primate_CI_high.dropna(), line = dict(width = 1, color = plotly.colors.DEFAULT_PLOTLY_COLORS[4]), legendgroup = 'primate', showlegend=False))
    fig.add_trace(go.Scatter(x = primate_CI_low.dropna().index, y= primate_CI_low.dropna(), line = dict(width = 1, color = plotly.colors.DEFAULT_PLOTLY_COLORS[4]), fill = 'tonexty', fillcolor = 'rgba(' + plotly.colors.DEFAULT_PLOTLY_COLORS[4][4:-1] + ', 0.2)', legendgroup = 'primate', name = 'primate range'))

    fig.add_trace(go.Scatter(x = bootstrap_counts_mean[motif].index * len(motif), y = bootstrap_counts_mean[motif], line = dict(width = 4, color = 'rgba(0,0,0,0.8)'), legendgroup = 'ci', name = 'T2T-CHM13'))

    fig.update_yaxes(type = 'log', title = 'counts <i>(normalized counts)</i>', range = [-0.05,9.05], ticktext = ['<b>1e8</b><br><i>(8e-2)</i>', '<b>1e6</b><br><i>(8e-4)</i>', '<b>1e4</b><br><i>(8e-6)</i>', '<b>1e2</b><br><i>(8e-8)</i>', '<b>1</b><br><i>(8e-10) </i>'], tickvals = [1e8, 1e6, 1e4, 1e2, 1])
#    fig.update_yaxes(type = 'log', title = '<b>counts</b> <i>(normalized counts)</i>', range = [-0.05,9.05], ticktext = ['<i>(9e-1)</i>', '<b>1e8</b>', '<i>(8e-3)</i>', '<b>1e6</b>', '<i>(8e-5)</i>', '<b>1e4</b>', '<i>(8e-7)</i>', '<b>1e2</b>', '<i>(7e-9)</i>', '<b>1</b>'], tickvals = [1e9,1e8,1e7,1e6,1e5,1e4,1e3,1e2,1e1,1])
    fig.update_xaxes(type = 'log', title = 'repeat tract length (units)', range = [-0.005,2.33], tickvals = [1,2,5,10,20,50,100,200], gridcolor = 'rgba(0,0,0,0.2)')
    fig.update_yaxes(gridcolor = 'rgba(0,0,0,0.2)')
    fig.update_layout(width = 1000, height = 500, font=dict(family = 'Arial', size = 20), margin={'t':20,'l':100,'b':50,'r':10})
    return fig

In [ ]:
def make_fig_rates_list_contour(contour, rates = rates_all, show_decode = True, priorname = 'uniform'):
    fig_rates = go.Figure()
    # denovo
    fig_rates.add_trace(go.Scatter(x = denovo_exp_rate['A'].loc[:8].index, y = denovo_exp_rate['A'].loc[:8], mode = 'markers', marker = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], symbol = 'square-open', line_width = 4, size = 8), name = 'Exp. (pooled trios)'))
    fig_rates.add_trace(go.Scatter(x = denovo_con_rate['A'].loc[:8].index, y = denovo_con_rate['A'].loc[:8], mode = 'markers', marker = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[1], symbol = 'square-open', line_width = 4, size = 8), name = 'Con. (pooled trios)'))
    # deCODE
    if show_decode == True:
        fig_rates.add_trace(go.Scatter(x = decode_exp_rate['A'].index, y = decode_exp_rate['A'], mode = 'markers', marker = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], symbol = 'circle-open', line_width = 4, size = 8), name = 'Exp./ins. (popSTR)'))
        fig_rates.add_trace(go.Scatter(x = decode_con_rate['A'].index, y = decode_con_rate['A'], mode = 'markers', marker = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[1], symbol = 'circle-open', line_width = 4, size = 8), name = 'Con. (popSTR)'))
    

    if contour is not None:
        current_contour = contour.loc[contour['level_0'] == 1].copy().reset_index()
        current_contour['exp_i'] = intercept_revlist.reindex(current_contour['exp_i']).values
        current_contour['con_i'] = intercept_revlist.reindex(current_contour['con_i']).values
        current_contour.set_index(['exp_i', 'con_i', 'exp_p', 'con_p'], inplace = True)
        current_exp = pd.concat(list(rates[1].reindex(current_contour.index)), axis=1)
        current_con = pd.concat(list(rates[2].reindex(current_contour.index)), axis=1)
    
        fig_rates.add_trace(go.Scatter(x = (current_exp.index), y = current_exp.min(axis=1), mode = 'lines', line = dict(width = 0, color = plotly.colors.DEFAULT_PLOTLY_COLORS[0]), legendgroup = 'exp', name = 'exp', showlegend = False))
        fig_rates.add_trace(go.Scatter(x = (current_exp.index), y = current_exp.max(axis=1), fill = 'tonexty', fillcolor = 'rgba(' + plotly.colors.DEFAULT_PLOTLY_COLORS[0][4:-1] + ', 0.2)', mode = 'lines', line = dict(width = 0, color = plotly.colors.DEFAULT_PLOTLY_COLORS[0]), legendgroup = 'exp', name = '𝜀 95% HDR (' + priorname + ')'))
    
        fig_rates.add_trace(go.Scatter(x = (current_con.index), y = current_con.min(axis=1), mode = 'lines', line = dict(width = 0, color = plotly.colors.DEFAULT_PLOTLY_COLORS[1]), legendgroup = 'con', name = 'con', showlegend = False))
        fig_rates.add_trace(go.Scatter(x = (current_con.index), y = current_con.max(axis=1), fill = 'tonexty', fillcolor = 'rgba(' + plotly.colors.DEFAULT_PLOTLY_COLORS[1][4:-1] + ', 0.2)', mode = 'lines', line = dict(width = 0, color = plotly.colors.DEFAULT_PLOTLY_COLORS[1]), legendgroup = 'con', name = '𝜅 95% HDR (' + priorname + ')'))
   
    fig_rates.update_yaxes(type = 'log', title = 'rate (per nt per generation)', range = [-12, -3], tickformat = '1.0e', dtick = 2)
    fig_rates.update_xaxes(type = 'log', title = 'repeat tract length (units)', range = [-0.05,2.33], tickvals = [1,2,5,10,20,50,100,200], gridcolor = 'rgba(0,0,0,0.2)')
    fig_rates.update_yaxes(gridcolor = 'rgba(0,0,0,0.2)')
    fig_rates.update_layout(width = 900, height = 500, font=dict(family = 'Arial', size = 20), margin={'t':20,'l':100,'b':50,'r':10})
    return fig_rates

#### 3 parameter multiplier-coupled power-law model 

In [ ]:
# Fig. 5b
current_model = 'PL'; current_prior = 'uniform'; current_param = '3param_Mult'
current_posterior = posteriors_all[current_model][current_prior][current_param].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])

fig = make_fig_list_contour(inference_grid_final_nointerp_subonly_norm_ps, posterior_contour, norm_range=(range(4,201)), showgroups=[1], tracename = '95% HDR (uniform prior)')
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg[current_model].loc[current_prior].loc[current_param] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL (uninformative)', line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], width = 3)))
#fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg[current_model].T['permissive'][current_param] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL (permissive)', line = dict(width = 3, dash = 'dash', color = plotly.colors.DEFAULT_PLOTLY_COLORS[0])))
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg[current_model].T['restrictive'][current_param] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL (informative)', line = dict(width = 3, dash = 'dot', color = plotly.colors.DEFAULT_PLOTLY_COLORS[0])))

In [ ]:
fig.write_image('plots/DRL_95HDR_PL_uniform_3mult.pdf')

In [ ]:
# Fig. S7b (1of3)
current_model = 'PL'; current_prior = 'uniform'; current_param = '3param_Mult'
current_posterior = posteriors_all[current_model][current_prior][current_param].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])

fig = make_fig_list_contour(inference_grid_final_nointerp_subonly_norm_ps, posterior_contour, norm_range=(range(4,201)), showgroups=[3,2,1,0])
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg[current_model].loc[current_prior].loc[current_param] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL<br>(uniform)', line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[3], dash = 'dash', width = 4)))
fig.update_layout(width = 800, height = 500)

In [ ]:
fig.write_image('plots/DRL_CDFbands_PL_uniform_3mult.pdf')

In [ ]:
# Fig. S7b (2of3)
current_model = 'PL'; current_prior = 'permissive'; current_param = '3param_Mult'
current_posterior = posteriors_all[current_model][current_prior][current_param].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])

fig = make_fig_list_contour(inference_grid_final_nointerp_subonly_norm_ps, posterior_contour, norm_range=(range(4,201)), showgroups=[3,2,1,0])
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg[current_model].loc[current_prior].loc[current_param] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL<br>(permissive)', line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[3], dash = 'dash', width = 4)))
fig.update_layout(width = 800, height = 500)

In [ ]:
fig.write_image('plots/DRL_CDFbands_PL_permissive_3mult.pdf')

In [ ]:
# Fig. S7b (3of3)
current_model = 'PL'; current_prior = 'restrictive'; current_param = '3param_Mult'
current_posterior = posteriors_all[current_model][current_prior][current_param].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])

fig = make_fig_list_contour(inference_grid_final_nointerp_subonly_norm_ps, posterior_contour, norm_range=(range(4,201)), showgroups=[3,2,1,0])
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg[current_model].loc[current_prior].loc[current_param] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL<br>(restrictive)', line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[3], dash = 'dash', width = 4)))
fig.update_layout(width = 800, height = 500)

In [ ]:
fig.write_image('plots/DRL_CDFbands_PL_restrictive_3mult.pdf')

In [ ]:
# Fig. 5c
fig_rates = make_fig_rates_list_contour(None, rates_all)
for prior, priorname, dash in zip(['uniform', 'restrictive'], ['uninformative', 'informative'], [None, 'dot']):
    fig_rates.add_trace(go.Scatter(x = list(range(9,200)), y = rates_weighted_exp[current_model].loc[prior][current_param].reindex(range(9,200)), mode = 'lines', legendgroup = prior, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], dash = dash, width = 4), name = 'weighted 𝜀|<i>L</i> (' + priorname + ')',))
    fig_rates.add_trace(go.Scatter(x = list(range(9,200)), y = rates_weighted_con[current_model].loc[prior][current_param].reindex(range(9,200)), mode = 'lines', legendgroup = prior, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[1], dash = dash, width = 4), name = 'weighted 𝜅|<i>L</i> (' + priorname + ')',))
fig_rates.show()

In [ ]:
fig_rates.write_image('plots/rates_95c_weighted_PL_allpriors_3mult.pdf')

#### 3 parameter multiplier-coupled power-law model with interpolation

In [ ]:
# Fig. S8b
current_model = 'PLinterp'; current_prior = 'uniform'; current_param = '3param_Mult'
current_posterior = posteriors_all[current_model][current_prior][current_param].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])

fig = make_fig_list_contour(inference_grid_final_interp_subonly_norm_ps, posterior_contour, norm_range=(range(4,201)), showgroups=[1], tracename = '95% HDR (uniform prior)')
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg[current_model].loc[current_prior].loc[current_param] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL (uniform)', line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], width = 3)))
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg[current_model].T['permissive'][current_param] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL (permissive)', line = dict(width = 3, dash = 'dash', color = plotly.colors.DEFAULT_PLOTLY_COLORS[0])))
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg[current_model].T['restrictive'][current_param] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL (restrictive)', line = dict(width = 3, dash = 'dot', color = plotly.colors.DEFAULT_PLOTLY_COLORS[0])))

In [ ]:
fig.write_image('plots/DRL_95HDR_PLinterp_uniform_3mult.pdf')

In [ ]:
# Fig. S8c
fig_rates = make_fig_rates_list_contour(None, rates_all)
for prior, dash in zip(['uniform', 'permissive', 'restrictive'], [None, 'dash', 'dot']):
    fig_rates.add_trace(go.Scatter(x = list(range(9,200)), y = rates_weighted_exp[current_model].loc[prior][current_param].reindex(range(9,200)), mode = 'lines', legendgroup = prior, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], dash = dash, width = 4), name = 'weighted 𝜀|<i>L</i> (' + prior + ')',))
    fig_rates.add_trace(go.Scatter(x = list(range(9,200)), y = rates_weighted_con[current_model].loc[prior][current_param].reindex(range(9,200)), mode = 'lines', legendgroup = prior, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[1], dash = dash, width = 4), name = 'weighted 𝜅|<i>L</i> (' + prior + ')',))
fig_rates.show()

In [ ]:
fig_rates.write_image('plots/rates_95c_weighted_PLinterp_allpriors_3mult.pdf')

#### 4 parameter power-law model

In [ ]:
# Fig. 10b
current_model = 'PL'; current_prior = 'uniform'; current_param = '4param'
current_posterior = posteriors_all[current_model][current_prior][current_param].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['exp_i', 'con_i', 'exp_p', 'con_p'])

fig = make_fig_list_contour(inference_grid_final_nointerp_subonly_norm_ps, posterior_contour, norm_range=(range(4,201)), showgroups=[1], tracename = '95% HDR (uniform prior)')
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg[current_model].loc[current_prior].loc[current_param] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL (uniform)', line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], width = 3)))
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg[current_model].T['permissive'][current_param] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL (permissive)', line = dict(width = 3, dash = 'dash', color = plotly.colors.DEFAULT_PLOTLY_COLORS[0])))
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg[current_model].T['restrictive'][current_param] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL (restrictive)', line = dict(width = 3, dash = 'dot', color = plotly.colors.DEFAULT_PLOTLY_COLORS[0])))

In [ ]:
fig.write_image('plots/DRL_95HDR_PL_uniform_4param.pdf')

In [ ]:
# Fig. 10c
fig_rates = make_fig_rates_list_contour(None, rates_all)
for prior, dash in zip(['uniform', 'permissive', 'restrictive'], [None, 'dash', 'dot']):
    fig_rates.add_trace(go.Scatter(x = list(range(9,200)), y = rates_weighted_exp[current_model].loc[prior][current_param].reindex(range(9,200)), mode = 'lines', legendgroup = prior, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], dash = dash, width = 4), name = 'weighted 𝜀|<i>L</i> (' + prior + ')',))
    fig_rates.add_trace(go.Scatter(x = list(range(9,200)), y = rates_weighted_con[current_model].loc[prior][current_param].reindex(range(9,200)), mode = 'lines', legendgroup = prior, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[1], dash = dash, width = 4), name = 'weighted 𝜅|<i>L</i> (' + prior + ')',))
fig_rates.show()

In [ ]:
fig_rates.write_image('plots/rates_95c_weighted_PL_allpriors_4param.pdf')

## range of Lfis values (for Supplementary Note 1)

In [ ]:
Lfis_PL_uniform = posteriors_all['PL']['uniform']['3param_Mult'].copy()
Lfis_PL_uniform['Lfis'] = 9*((denovo_substitution_context_rate.loc['A']['Afission'] / (Lfis_PL_uniform['M'] * denovo_nonexp_rate['A'][8]))**(1/Lfis_PL_uniform['exp_p'])).replace(np.inf, np.nan)
Lfis_PL_uniform = Lfis_PL_uniform.dropna()
num95 = (posteriors_all['PL']['uniform']['3param_Mult'][0].sort_values(ascending = False).cumsum() < 0.95).reset_index(drop=True).idxmin()
Lfis_PL_uniform.sort_values(by = [0], ascending = False)[:num95]['Lfis'].median()
np.average(Lfis_PL_uniform['Lfis'], weights = Lfis_PL_uniform[0]), Lfis_PL_uniform.sort_values(by = [0], ascending = False)[:num95]['Lfis'].median()

In [ ]:
 Lfis_PL_uniform.sort_values(by = [0], ascending = False)[:num95]['Lfis'].min(),  Lfis_PL_uniform.sort_values(by = [0], ascending = False)[:num95]['Lfis'].max()

In [ ]:
Lfis_PL_permissive = posteriors_all['PL']['permissive']['3param_Mult'].copy()
Lfis_PL_permissive['Lfis'] = 9*((denovo_substitution_context_rate.loc['A']['Afission'] / (Lfis_PL_permissive['M'] * denovo_nonexp_rate['A'][8]))**(1/Lfis_PL_permissive['exp_p'])).replace(np.inf, np.nan)
Lfis_PL_permissive = Lfis_PL_permissive.dropna()
num95 = (posteriors_all['PL']['permissive']['3param_Mult'][0].sort_values(ascending = False).cumsum() < 0.95).reset_index(drop=True).idxmin()
np.average(Lfis_PL_permissive['Lfis'], weights = Lfis_PL_permissive[0]), Lfis_PL_permissive.sort_values(by = [0], ascending = False)[:num95]['Lfis'].median()

In [ ]:
 Lfis_PL_permissive.sort_values(by = [0], ascending = False)[:num95]['Lfis'].min(),  Lfis_PL_permissive.sort_values(by = [0], ascending = False)[:num95]['Lfis'].max()

In [ ]:
Lfis_PL_restrictive = posteriors_all['PL']['restrictive']['3param_Mult'].copy()
Lfis_PL_restrictive['Lfis'] = 9*((denovo_substitution_context_rate.loc['A']['Afission'] / (Lfis_PL_restrictive['M'] * denovo_nonexp_rate['A'][8]))**(1/Lfis_PL_restrictive['exp_p'])).replace(np.inf, np.nan)
Lfis_PL_restrictive = Lfis_PL_restrictive.dropna()
num95 = (posteriors_all['PL']['restrictive']['3param_Mult'][0].sort_values(ascending = False).cumsum() < 0.95).reset_index(drop=True).idxmin()
np.average(Lfis_PL_restrictive['Lfis'], weights = Lfis_PL_restrictive[0]), Lfis_PL_restrictive.sort_values(by = [0], ascending = False)[:num95]['Lfis'].median()

In [ ]:
 Lfis_PL_restrictive.sort_values(by = [0], ascending = False)[:num95]['Lfis'].min(),  Lfis_PL_restrictive.sort_values(by = [0], ascending = False)[:num95]['Lfis'].max()

## Constant speedup for 95% HDR of power-law model

#### Make jobs list for running computational model on cluster

In [ ]:
# gather all parameters in the 95% HDR
current_posterior = posteriors_all['PL']['uniform']['3param_Mult'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
hdr95_uniform = posterior_contour.loc[posterior_contour['level_0'] <2]

current_posterior = posteriors_all['PL']['permissive']['3param_Mult'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
hdr95_permissive = posterior_contour.loc[posterior_contour['level_0'] <2]

current_posterior = posteriors_all['PL']['restrictive']['3param_Mult'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
hdr95_restrictive = posterior_contour.loc[posterior_contour['level_0'] <2]

hdr_all = pd.concat([hdr95_uniform, hdr95_permissive, hdr95_restrictive]).reset_index()[['exp_i', 'con_i', 'exp_p', 'con_p']]
hdr_all['exp_i'] = intercept_revlist.reindex(hdr_all['exp_i']).values
hdr_all['con_i'] = intercept_revlist.reindex(hdr_all['con_i']).values
hdr_all = hdr_all.sort_values(by = ['exp_i', 'con_i', 'exp_p', 'con_p'])

hdr_all = pd.Series([(a,b,c,d) for a,b,c,d in zip(hdr_all['exp_i'], hdr_all['con_i'], hdr_all['exp_p'], hdr_all['con_p'])])
hdr_all = hdr_all.drop_duplicates()

jobs_group = dict()
for n in range(round(len(hdr_all)/20)+1):
    jobs_group[n] = hdr_all[n*20:n*20+20]
jobs_group = pd.concat(jobs_group)

In [ ]:
jobs_group.to_pickle('95hdr_all.pickle')

#### Load data files
- note: pause here and run computational model script for the parameter combinations determined above
- run 'gather_grid' script to collect this data prior to running below section

In [ ]:
grid_95c_constantspeedup_subonly = pd.read_pickle('simulations/completed_grids/top95constant_subonlyrerun.pickle')
grid_95c_constantspeedup_subonly_final = pd.read_pickle('simulations/completed_grids/top95constant_subonlyrerun_final.pickle').droplevel(4, axis=1)#.dropna(how = 'all', axis=1)

In [ ]:
# Normalize counts (with pseudocount)
grid_95c_constantspeedup_subonly_final_norm_ps = (grid_95c_constantspeedup_subonly_final.reindex(norm_range).fillna(0) + pseudocount) / (grid_95c_constantspeedup_subonly_final.reindex(norm_range).fillna(0) + pseudocount).sum()
grid_95c_constantspeedup_subonly_norm_ps = (grid_95c_constantspeedup_subonly.reindex(norm_range).fillna(0) + pseudocount) / (grid_95c_constantspeedup_subonly.reindex(norm_range).fillna(0) + pseudocount).sum()

#### Calculate KL for a given timepoint compared to the final timepoint

In [ ]:
def calc_KL_divergence_ab(dist_a, dist_b, L_range = range(4,201)):
    return ((dist_a.reindex(L_range)) * np.log((dist_a.reindex(L_range)) / (dist_b.reindex(L_range)))).sum()

KLtoend_inference_grid_subonly_timeseries = dict()
for time in grid_95c_constantspeedup_subonly_norm_ps.columns.levels[4]:
    KLtoend_inference_grid_subonly_timeseries[time] = calc_KL_divergence_ab(grid_95c_constantspeedup_subonly_norm_ps.xs(1e9, 1, level=4), grid_95c_constantspeedup_subonly_norm_ps.xs(time, 1, level=4), L_range=L_range)
KLtoend_inference_grid_subonly_timeseries = pd.concat(KLtoend_inference_grid_subonly_timeseries, axis=1)
KLtoend_inference_grid_subonly_timeseries = KLtoend_inference_grid_subonly_timeseries.replace(0, np.nan)
KLtoend_inference_grid_subonly_timeseries[1e9] = 0

In [ ]:
def make_colorscale(list_of_traces, opacity = 0.15):
    current_colors = pd.Series(['rgb('+str(current/len(list_of_traces)*255) + ', 180, '+str((len(list_of_traces)-current)/len(list_of_traces)*255)+')' for current in range(len(list_of_traces))], index = list_of_traces)
    return pd.Series(current_colors, index = list_of_traces), pd.Series(['rgba' + color[3:-1] + ', '+str(opacity)+')' for color in current_colors], index = list_of_traces)
dtau_range = list(pd.Series(list(grid_95c_constantspeedup_subonly_norm_ps.columns.get_level_values(3) - grid_95c_constantspeedup_subonly_norm_ps.columns.get_level_values(2))).round(2).value_counts().sort_index().index)
dtau_range_colorscale = make_colorscale(dtau_range[::-1])

In [ ]:
timepoint_min = KLtoend_inference_grid_subonly_timeseries.fillna(np.inf).lt((0.05* KLtoend_inference_grid_subonly_timeseries.max(axis=1)), axis=0).idxmax(axis=1)

In [ ]:
# Fig. S10b (top left)
fig_timescale = go.Figure()
counter = 0
for param in KLtoend_inference_grid_subonly_timeseries.index:
    fig_timescale.add_trace(go.Scatter(x = KLtoend_inference_grid_subonly_timeseries.loc[param].dropna().index, y = KLtoend_inference_grid_subonly_timeseries.loc[param].dropna(), line = dict(width = 4, color = dtau_range_colorscale[1][round(param[3] - param[2], 2)]), legendgroup = round(param[3] - param[2], 2), name = str(param), text = str(param), showlegend = False))
    counter +=1
fig_timescale.add_trace(go.Scatter(x = timepoint_min, y = [KLtoend_inference_grid_subonly_timeseries.loc[param].loc[time] for param, time in zip(KLtoend_inference_grid_subonly_timeseries.index, timepoint_min)], mode = 'markers', marker = dict(size = 4, color = 'blue'), name = '< 5% of initial KL', text = KLtoend_inference_grid_subonly_timeseries.index))
fig_timescale.update_xaxes(title = 'time (generations)', range = [0, 1.001e9])#, type = 'log')
fig_timescale.update_yaxes(title = 'KL (vs. final time point)')#, type = 'log')
fig_timescale.update_layout(font=dict(family = 'Arial', size = 16), height = 320, width = 500, margin={'t':10,'l':60,'b':45,'r':10}, legend=dict(yanchor="top", y=0.35, xanchor="right", x=0.6, borderwidth = 2))

fig_timescale.add_trace(go.Scatter(x=[None], y=[None], showlegend = False, mode='markers', marker=dict(colorscale=dtau_range_colorscale[0].values, showscale=True,
                                 colorbar=dict(len = 0.75, yanchor = 'bottom', y = 0.2, title = 'Δτ', thickness=5, tickvals = np.linspace(6.75,1.25,6), ticktext=[-0.4, -0.2, 0, 0.2, 0.4, 0.6], outlinewidth=0))))


fig_timescale.show()

In [ ]:
fig_timescale.write_image('plots/fig_timescale_95c.pdf')

In [ ]:
# Fig. S10b (top right)
fig_timescale = go.Figure()
counter = 0
for param in KLtoend_inference_grid_subonly_timeseries.index:
    fig_timescale.add_trace(go.Scatter(x = KLtoend_inference_grid_subonly_timeseries.loc[param].dropna().index[:-1], y = KLtoend_inference_grid_subonly_timeseries.loc[param][:-1].dropna(), line = dict(width = 4, color = dtau_range_colorscale[1][round(param[3] - param[2], 2)]), legendgroup = round(param[3] - param[2], 2), name = str(param), text = str(param), showlegend = False))
    counter +=1
fig_timescale.add_trace(go.Scatter(x = timepoint_min, y = [KLtoend_inference_grid_subonly_timeseries.loc[param].loc[time] for param, time in zip(KLtoend_inference_grid_subonly_timeseries.index, timepoint_min)], mode = 'markers', marker = dict(size = 4, color = 'blue'), name = '< 5% of initial KL', text = KLtoend_inference_grid_subonly_timeseries.index))
fig_timescale.update_xaxes(title = 'time (generations)', range = [0, 1.001e9])#, type = 'log')
fig_timescale.update_yaxes(title = 'KL (vs. final time point)', type = 'log', range = [-6,-1])
fig_timescale.update_layout(font=dict(family = 'Arial', size = 16), height = 320, width = 500, margin={'t':10,'l':60,'b':45,'r':10}, legend=dict(yanchor="top", y=0.2, xanchor="right", x=0.6, borderwidth = 2))
fig_timescale.show()

In [ ]:
fig_timescale.write_image('plots/fig_timescale_95c_log.pdf')

In [ ]:
# Fig. S10b (bottom right)
colorscale_time = pd.Series(plotly.colors.sample_colorscale('plasma_r', 7, low = 0.1, high = 0.9), index = [1e6, 3e6, 1e7, 3e7, 1e8, 3e8, 1e9])

def make_evolve_fig(input_df, motif = 'A', showlegend = True):
    fig = go.Figure()
    reps = [1e6, 3e6, 1e7, 3e7, 1e8, 3e8, 1e9]
    
    fig.add_trace(go.Scattergl(x = input_df.index, y = input_df[0] / input_df[0].loc[2:].sum() * bootstrap_counts_mean[motif].loc[2:].sum(), name = 'initial state', legendgroup = 'start', connectgaps = True, line = dict(color = 'gray', dash = 'dash', width = 2.5)))
    for rep in reps:
        fig.add_trace(go.Scattergl(x = input_df.index, y = input_df[rep] / input_df[rep].loc[4:].sum() * bootstrap_counts_mean[motif].loc[4:].sum(), name = '{:0.1e}'.format(rep), showlegend = showlegend, line = dict(color = colorscale_time[rep], width = 3), opacity = 0.7))
    fig.add_trace(go.Scatter(x = bootstrap_counts_mean[motif].index, y = bootstrap_counts_mean[motif], line = dict(color = 'rgba(0,0,0,0.8)'), legendgroup = 'ci', name = 'T2T-CHM13'))
    fig.add_trace(go.Scatter(x = bootstrap_counts_max[motif].index, y = bootstrap_counts_max[motif], line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 95%'))
    fig.add_trace(go.Scatter(x = bootstrap_counts_min[motif].index, y = bootstrap_counts_min[motif], fill='tonexty', fillcolor = 'rgba(0,0,0,0.25)', line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 5%'))
    fig.update_xaxes(type = 'log', title = 'repeat length (units)', range = [0,2])
    fig.update_yaxes(type = 'log', title = 'repeat counts', range = [-0.05,9.05], tickformat = '1.0e', dtick = 2)
    fig.update_layout(font=dict(family = 'Arial', size = 16), height = 320, width = 800, margin={'t':10,'l':80,'b':55,'r':10})      
    return fig

fig_evolve = make_evolve_fig(grid_95c_constantspeedup_subonly[3][2][1.6][2], showlegend = False)
fig_evolve.update_xaxes(tickvals = [1,2,5,10,20,50,80])
fig_evolve.add_trace(go.Scatter(x=[None], y=[None], showlegend = False, mode='markers', marker=dict(colorscale=colorscale_time.values, showscale=True,
                                 colorbar=dict(len = 0.75, yanchor = 'bottom', y = 0, title = 'generation', thickness=5, tickvals = np.linspace(6.75,0,4), ticktext=['1e6', '1e7', '1e8', '1e9'][::-1], outlinewidth=0)), hoverinfo='none'))
fig_evolve.update_xaxes(title = 'repeat tract length (units)')
fig_evolve.update_yaxes(title = 'counts')

In [ ]:
fig_evolve.write_image('plots/fig_timescale_evolve_bestpoint.pdf')

In [ ]:
# Fig. S10b (bottom left)
round_localmin = timepoint_min.reset_index()
round_localmin.columns = ['exp_i',  'con_i', 'exp_p', 'con_p', 'round']
round_localmin['M'] = pd.Series(m_list).reindex(round_localmin['exp_i']-1).values
round_localmin = round_localmin.set_index(['M', 'exp_p', 'con_p'])['round']

fig = make_subplots(rows = 1, cols = len(round_localmin.index.levels[0]), shared_xaxes = True, shared_yaxes = True, horizontal_spacing = 0.04, subplot_titles = ['<i>m</i>=' + str(round(x / intercept_list[1], 1)) for x in intercept_list[2:6]], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')
row_count=1; col_count = 0
for m in round_localmin.index.levels[0]:
    col_count+=1
    if m in round_localmin.index.get_level_values('M'):
        fig.add_trace(go.Heatmap(x = round_localmin.loc[m].reindex(inference_grid_final_nointerp_subonly[0][0].columns).index.get_level_values(0), y = round_localmin.loc[m].reindex(inference_grid_final_nointerp_subonly[0][0].columns).index.get_level_values(1), z=(round_localmin.loc[m].reindex(inference_grid_final_nointerp_subonly[0][0].columns)), coloraxis='coloraxis1'), row = row_count, col = col_count)
fig.update_layout(coloraxis1=dict(colorscale='plasma_r', colorbar = dict(title = 'generation', len=1.9), cmin = 1.8e7, cmax = round_localmin.max()))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 200, width = 600, margin={'t':50,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)

fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)
fig.show()

In [ ]:
fig.write_image('plots/fig_timescale_95c_roundlocalmin.pdf')

# Robustness checks

## Constant speedup (sparse grid)

### From geometric starting state

In [ ]:
# Normalize with pseudocount
inference_grid_final_sparse_constantspeedup_subonly_norm_PS = (inference_grid_final_sparse_constantspeedup_subonly.fillna(0) + pseudocount).reindex(L_range) / (inference_grid_final_sparse_constantspeedup_subonly.fillna(0) + pseudocount).reindex(L_range).sum()
# Calculate KL
KL_grid_sparse_constantspeedup_subonly = inference_grid_final_sparse_constantspeedup_subonly_norm_PS.apply(lambda y: calc_KL_divergence_CHM13(y, L_range=L_range)).reset_index()
# 3M grid with spacing 0.2
KL_grid_sparse_constantspeedup_subonly = KL_grid_sparse_constantspeedup_subonly.loc[(KL_grid_sparse_constantspeedup_subonly['exp_i'] == 1+ KL_grid_sparse_constantspeedup_subonly['con_i']) & (KL_grid_sparse_constantspeedup_subonly['exp_p'].isin([n/10 for n in range(0,41,2)])) & (KL_grid_sparse_constantspeedup_subonly['con_p'].isin([n/10 for n in range(0,41,2)]))].set_index(['exp_i', 'con_i', 'exp_p', 'con_p'])
# Calculate acceptance probability
grid_sparse_constantspeedup_subonly_likelihood = calc_likelihood_from_primate_metric_KL(KL_grid_sparse_constantspeedup_subonly)

In [ ]:
bayesfactors['constant'] = dict(); best_params['constant'] = dict(); avg_params['constant'] = dict(); drl_weighted_avg['constant'] = dict(); posteriors_all['constant'] = dict(); posterior_95c['constant'] = dict(); grid_drl_95c['constant'] = dict(); rates_weighted_exp['constant'] = dict(); rates_weighted_con['constant'] = dict(); grid_rates_95c['constant'] = dict(); subset_indices['constant'] = dict()
# uniform prior
bayesfactors['constant']['uniform'] = dict(); best_params['constant']['uniform'] = dict(); avg_params['constant']['uniform'] = dict(); drl_weighted_avg['constant']['uniform'] = dict(); posteriors_all['constant']['uniform'] = dict(); posterior_95c['constant']['uniform'] = dict(); grid_drl_95c['constant']['uniform'] = dict(); rates_weighted_exp['constant']['uniform'] = dict(); rates_weighted_con['constant']['uniform'] = dict(); grid_rates_95c['constant']['uniform'] = dict()

subset_indices['constant']['3param_Mult'] = grid_sparse_constantspeedup_subonly_likelihood.index
model_summary(grid_sparse_constantspeedup_subonly_likelihood, 'constant', 'uniform', '3param_Mult', prior = None, params = ['M', 'exp_p', 'con_p'])

avg_params['constant']['uniform'] = pd.concat(avg_params['constant']['uniform']).unstack()
avg_params['constant'] = pd.concat(avg_params['constant'])
best_params['constant']['uniform'] = pd.concat(best_params['constant']['uniform']).unstack()
best_params['constant'] = pd.concat(best_params['constant'])
drl_weighted_avg['constant']['uniform'] = pd.concat(drl_weighted_avg['constant']['uniform']).unstack()
drl_weighted_avg['constant'] = pd.concat(drl_weighted_avg['constant'])

In [ ]:
current_model = 'constant'; current_prior = 'uniform'; current_param = '3param_Mult'
current_posterior = posteriors_all[current_model][current_prior][current_param].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour_3M_constant_sparse = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
# max KL value in 95% HDR
KL_95c_constant = KL_grid_sparse_constantspeedup_subonly[0].sort_values()[:len(posterior_contour_3M_constant_sparse.loc[posterior_contour_3M_constant_sparse['level_0'] <2][0])].max()

### Sparse grid subset for progressive speedup

In [ ]:
# Retrieve sparse grid subset
current_posterior = posteriors_all['PL']['uniform']['3param_Mult'].loc[(posteriors_all['PL']['uniform']['3param_Mult']['exp_p'].isin([n/10 for n in range(0,41,2)])) & (posteriors_all['PL']['uniform']['3param_Mult']['con_p'].isin([n/10 for n in range(0,41,2)]))].set_index(['exp_i', 'con_i', 'exp_p', 'con_p']).sort_values(by = [0], ascending = False)
# Normalize posterior
current_posterior[0] = current_posterior[0] / current_posterior[0].sum()
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour_3M_pro_sparse = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
# max KL value in 95% HDR
KL_95c_prospeedup = KL_inference_grid_final_nointerp.reindex(KL_grid_sparse_constantspeedup_subonly.index).sort_values()[:len(posterior_contour_3M_pro_sparse.loc[posterior_contour_3M_pro_sparse['level_0'] <2])].max()

### From uniform-like starting state

In [ ]:
# Normalize with pseudocount
inference_grid_final_sparse_constantspeedup_uniform_norm_PS = (inference_grid_final_sparse_constantspeedup_uniform.fillna(0) + pseudocount).reindex(L_range) / (inference_grid_final_sparse_constantspeedup_uniform.fillna(0) + pseudocount).reindex(L_range).sum()
# Calculate KL
KL_grid_sparse_constantspeedup_uniform = inference_grid_final_sparse_constantspeedup_uniform_norm_PS.apply(lambda y: calc_KL_divergence_CHM13(y, L_range=L_range)).reset_index()
# 3M grid with spacing 0.2
KL_grid_sparse_constantspeedup_uniform = KL_grid_sparse_constantspeedup_uniform.loc[(KL_grid_sparse_constantspeedup_uniform['exp_i'] == 1+ KL_grid_sparse_constantspeedup_uniform['con_i']) & (KL_grid_sparse_constantspeedup_uniform['exp_p'].isin([n/10 for n in range(0,41,2)])) & (KL_grid_sparse_constantspeedup_uniform['con_p'].isin([n/10 for n in range(0,41,2)]))].set_index(['exp_i', 'con_i', 'exp_p', 'con_p'])
# Calculate acceptance probability
grid_sparse_constantspeedup_uniform_likelihood = calc_likelihood_from_primate_metric_KL(KL_grid_sparse_constantspeedup_uniform).reindex(grid_sparse_constantspeedup_subonly_likelihood.index).fillna(0)

In [ ]:
bayesfactors['constant-uni'] = dict(); best_params['constant-uni'] = dict(); avg_params['constant-uni'] = dict(); drl_weighted_avg['constant-uni'] = dict(); posteriors_all['constant-uni'] = dict(); posterior_95c['constant-uni'] = dict(); grid_drl_95c['constant-uni'] = dict(); rates_weighted_exp['constant-uni'] = dict(); rates_weighted_con['constant-uni'] = dict(); grid_rates_95c['constant-uni'] = dict(); subset_indices['constant-uni'] = dict()
# uniform prior
bayesfactors['constant-uni']['uniform'] = dict(); best_params['constant-uni']['uniform'] = dict(); avg_params['constant-uni']['uniform'] = dict(); drl_weighted_avg['constant-uni']['uniform'] = dict(); posteriors_all['constant-uni']['uniform'] = dict(); posterior_95c['constant-uni']['uniform'] = dict(); grid_drl_95c['constant-uni']['uniform'] = dict(); rates_weighted_exp['constant-uni']['uniform'] = dict(); rates_weighted_con['constant-uni']['uniform'] = dict(); grid_rates_95c['constant-uni']['uniform'] = dict()

subset_indices['constant-uni']['3param_Mult'] = grid_sparse_constantspeedup_uniform_likelihood.index
model_summary(grid_sparse_constantspeedup_uniform_likelihood, 'constant-uni', 'uniform', '3param_Mult', prior = None, params = ['M', 'exp_p', 'con_p'])

avg_params['constant-uni']['uniform'] = pd.concat(avg_params['constant-uni']['uniform']).unstack()
avg_params['constant-uni'] = pd.concat(avg_params['constant-uni'])
best_params['constant-uni']['uniform'] = pd.concat(best_params['constant-uni']['uniform']).unstack()
best_params['constant-uni'] = pd.concat(best_params['constant-uni'])
drl_weighted_avg['constant-uni']['uniform'] = pd.concat(drl_weighted_avg['constant-uni']['uniform']).unstack()
drl_weighted_avg['constant-uni'] = pd.concat(drl_weighted_avg['constant-uni'])

In [ ]:
current_model = 'constant-uni'; current_prior = 'uniform'; current_param = '3param_Mult'
current_posterior = posteriors_all[current_model][current_prior][current_param].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour_3M_constant_uniform_sparse = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
# max KL value in 95% HDR
KL_95c_constant_uniform = KL_grid_sparse_constantspeedup_uniform[0].sort_values()[:len(posterior_contour_3M_constant_uniform_sparse.loc[posterior_contour_3M_constant_uniform_sparse['level_0'] <2][0])].max()

### Plot comparisons

#### Posterior space

In [ ]:
fig = make_subplots(rows = 3, cols = 7, shared_xaxes = True, shared_yaxes = True, horizontal_spacing = 0.02, vertical_spacing = 0.08, subplot_titles = ['<i>m</i>=' + str(round(x / intercept_list[1], 1)) for x in intercept_list[1:]], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')
row_count=1; col_count = 0
for m in m_list:
    col_count+=1
    fig.add_trace(go.Heatmap(x = posterior_contour_3M_pro_sparse.loc[m].index.get_level_values(0), y = posterior_contour_3M_pro_sparse.loc[m].index.get_level_values(1), z=(posterior_contour_3M_pro_sparse.loc[m]['level_0']+1).fillna(5), coloraxis='coloraxis1'), row = row_count, col = col_count)
row_count=2; col_count = 0
for m in m_list:
    col_count+=1
    fig.add_trace(go.Heatmap(x = posterior_contour_3M_constant_sparse.loc[m].index.get_level_values(0), y = posterior_contour_3M_constant_sparse .loc[m].index.get_level_values(1), z=(posterior_contour_3M_constant_sparse .loc[m]['level_0']+1).fillna(5), coloraxis='coloraxis1'), row = row_count, col = col_count)
row_count=3; col_count = 0
for m in m_list:
    col_count+=1
    fig.add_trace(go.Heatmap(x = posterior_contour_3M_constant_uniform_sparse.loc[m].index.get_level_values(0), y = posterior_contour_3M_constant_uniform_sparse.loc[m].index.get_level_values(1), z=(posterior_contour_3M_constant_uniform_sparse.loc[m]['level_0']+1).fillna(5), coloraxis='coloraxis1'), row = row_count, col = col_count)
fig.update_layout(coloraxis1=dict(colorscale=discrete_deep_range, colorbar = dict(title = 'HDR', len=0.55, tickvals = [1.5,2.5,3.5,4.5], ticktext = ['68%', '95%', '99.7%', '99.99%'], x = 1.075, y = 0.825), cmin = 1, cmax = 5))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 420, width = 1000, margin={'t':50,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)

fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)

fig.show()

In [ ]:
fig.write_image('plots/posterior_sparse_prospeedup_constantspeedup_uniformstart.pdf')

#### Posterior-weighted DRLs

In [ ]:
fig = make_fig_list_contour(inference_grid_final_nointerp_subonly_norm_ps, None, norm_range=(range(4,201)), showgroups=[1], tracename = '95% HDR (uniform prior)')
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg['PL'].loc['uniform'].loc['3param_Mult'] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL<br>(step-wise speedup,<br>geometric initial state)', line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], width = 3)))
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg['constant'].loc['uniform'].loc['3param_Mult'] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL<br>(constant speedup,<br>geometric initial state)', line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], dash = 'dash', width = 3)))
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg['constant-uni'].loc['uniform'].loc['3param_Mult'] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL<br>(constant speedup,<br>compound initial state)', line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], dash = 'dot', width = 3)))

In [ ]:
fig.write_image('plots/compare_DRL_95c_pro_constant_uniformstart.pdf')

#### Comparisons of KL values

In [ ]:
fig_comp = go.Figure(go.Scatter(x = KL_grid_sparse_constantspeedup_subonly[0], y = KL_inference_grid_final_nointerp.reindex(KL_grid_sparse_constantspeedup_subonly.index), mode = 'markers', text = list(KL_grid_sparse_constantspeedup_subonly.index), marker = dict(color = 'rgba(31, 119, 180, 0.15)'), showlegend = False))
fig_comp.add_trace(go.Scatter(x = [0, 5], y = [0,5], mode = 'lines', line = dict(color = 'black', width = 1), showlegend = False))
fig_comp.add_trace(go.Scatter(y=[0, KL_95c_prospeedup, KL_95c_prospeedup, 0], x=[0, 0, KL_95c_constant, KL_95c_constant], mode = 'lines', line = dict(width = 3, dash = 'dash', color = discrete_deep[1][1]), opacity = 0.95, fill="toself", fillcolor = 'rgba(' + pd.DataFrame(discrete_deep)[1][1][4:-1] + ', 0.15)', showlegend = False))

fig_comp.update_yaxes(title = dict(text = 'step-wise speed-up (<i>D<sub>KL</sub></i>)', font = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0])), type = 'log', dtick = 1)
fig_comp.update_xaxes(title = 'constant speed-up (<i>D<sub>KL</sub></i>)', dtick = 1, type = 'log', scaleanchor="y", scaleratio=1)
fig_comp.update_layout(width = 400, height = 400, font=dict(family = 'Arial', size = 16), margin={'t':10,'l':60,'b':55,'r':10})

In [ ]:
fig_comp.write_image('plots/KL_constant_v_prospeedup.pdf')

In [ ]:
fig_comp = go.Figure(go.Scatter(y = KL_grid_sparse_constantspeedup_uniform[0], x = KL_grid_sparse_constantspeedup_subonly[0].reindex(KL_grid_sparse_constantspeedup_uniform.index), mode = 'markers', text = list(KL_grid_sparse_constantspeedup_uniform.index), marker = dict(color = 'rgba(31, 119, 180, 0.15)'), showlegend = False))
fig_comp.add_trace(go.Scatter(x = [0, 5], y = [0,5], mode = 'lines', line = dict(color = 'black', width = 1), showlegend = False))
fig_comp.add_trace(go.Scatter(y=[0, KL_95c_constant_uniform, KL_95c_constant_uniform, 0], x=[0, 0, KL_95c_constant, KL_95c_constant], mode = 'lines', line = dict(width = 3, dash = 'dash', color = discrete_deep[1][1]), opacity = 0.95, fill="toself", fillcolor = 'rgba(' + pd.DataFrame(discrete_deep)[1][1][4:-1] + ', 0.15)', showlegend = False))

fig_comp.update_yaxes(title = dict(text = 'compound initial state (<i>D<sub>KL</sub></i>)', font = dict(color = 'red')), type = 'log', dtick = 1)
fig_comp.update_xaxes(title = 'geometric initial state (<i>D<sub>KL</sub></i>)', dtick = 1, type = 'log', scaleanchor="y", scaleratio=1)
fig_comp.update_layout(width = 400, height = 400, font=dict(family = 'Arial', size = 16), margin={'t':10,'l':60,'b':55,'r':10})

In [ ]:
fig_comp.write_image('plots/KL_constant_subonly_v_uniform.pdf')

## Effect of stochastics
- note: pause here and run computational model script for the parameter combinations determined above
- run 'gather_grid' script to collect this data prior to running below section

In [ ]:
# Load data
stoch_all = dict()
for stoch in range(1,11):
    stoch_all[stoch] = pd.read_pickle('simulations/completed_grids/grid_top95c_stoch_'+str(stoch)+'.pickle').sort_index(axis=1)

In [ ]:
inference_grid_top95c_norm_PS_stoch = dict(); KL_grid_top95c_timepoints_stoch = dict(); KL_final_top95c_stoch = dict()
for stoch in range(1,11):
    # Normalize with pseudocount
    inference_grid_top95c_norm_PS_stoch[stoch] = (stoch_all[stoch].fillna(0) + pseudocount).reindex(L_range) / (stoch_all[stoch].fillna(0) + pseudocount).reindex(L_range).sum()
    # Calculate KL
    KL_grid_top95c_timepoints_stoch[stoch] = inference_grid_top95c_norm_PS_stoch[stoch].apply(lambda y: calc_KL_divergence_CHM13(y, L_range=L_range)).unstack()
    KL_final_top95c_stoch[stoch] = KL_grid_top95c_timepoints_stoch[stoch].stack().reindex([(param[0], param[1], param[2], param[3], stoch_all[stoch][param].columns.max()) for param in stoch_all[stoch].xs(0, 1, level=4).columns])
    KL_final_top95c_stoch[stoch] = KL_final_top95c_stoch[stoch].reset_index().set_index(['exp_i', 'con_i', 'exp_p', 'con_p'])

In [ ]:
fig_comp = go.Figure()
for stoch in range(1,11):
    fig_comp.add_trace(go.Scatter(x = KL_inference_grid_final_nointerp.reindex(KL_final_top95c_stoch[stoch].index), y = KL_final_top95c_stoch[stoch][0], mode = 'markers', marker = dict(size = 4, opacity = 0.3), text = KL_final_top95c_stoch[stoch].index, showlegend = False))
fig_comp.add_trace(go.Scatter(x = [0.0032, 0.0051], y = [0.0032, 0.0051], mode = 'lines', line = dict(color = 'black', width = 1), showlegend = False))
fig_comp.update_yaxes(title = 'stochastics (<i>D<sub>KL</sub></i>)')
fig_comp.update_xaxes(title = 'deterministic (<i>D<sub>KL</sub></i>)', scaleanchor="y", scaleratio=1)
fig_comp.update_layout(width = 400, height = 400, font=dict(family = 'Arial', size = 16), margin={'t':10,'l':90,'b':55,'r':10})
fig_comp.show()

In [ ]:
fig_comp.write_image('plots/KL_stochastics_top95c.pdf')

# Alternate parameterizations

## Pure power-law model

In [ ]:
# restrict to contraction-biased parameters
inference_grid_final_xrange = inference_grid_final_xrange.T.reset_index()
inference_grid_final_xrange = inference_grid_final_xrange.where(inference_grid_final_xrange['exp_p'] < inference_grid_final_xrange['con_p']).dropna(how = 'all').set_index(['exp_i', 'con_i', 'exp_p', 'con_p']).T

In [ ]:
# Normalize counts (with pseudocount)
inference_grid_final_xrange_norm_ps = (inference_grid_final_xrange.reindex(norm_range) + pseudocount) / (inference_grid_final_xrange.reindex(norm_range) + pseudocount).sum()
# Calculate KL
KL_inference_grid_final_xrange = inference_grid_final_xrange_norm_ps.apply(lambda y: calc_KL_divergence_CHM13(y, L_range=L_range))
# Calculate acceptance probability
grid_likelihood_xrange_KL = calc_likelihood_from_primate_metric_KL(KL_inference_grid_final_xrange)

#### grid of rates

In [ ]:
#### alternate rate functional forms
def power_law_rate_at_L(power, L, rate = 1e-8, bins = 200):
    return pd.Series([np.exp(np.log(rate) - power * np.log(L) + np.log(length)*power) for length in range(1,bins)], index = range(1,bins))
mu_nu_all = pd.read_pickle('denovo/mu_nu_1-3.pickle')
rates_all_xrange = pd.DataFrame([((exp_i, con_i, exp_p, con_p), power_law_rate_at_L(exp_p, exp_i, rate = mu_nu_all['mu'][1]['A']), power_law_rate_at_L(con_p, con_i, rate = mu_nu_all['nu'][1]['A'])) for (exp_i, con_i, exp_p, con_p) in inference_grid_final_xrange.columns], index = inference_grid_final_xrange.columns)

### Calculate Bayes factors, etc.

In [ ]:
bayesfactors['PL_only'] = dict(); best_params['PL_only'] = dict(); avg_params['PL_only'] = dict(); drl_weighted_avg['PL_only'] = dict(); posteriors_all['PL_only'] = dict(); posterior_95c['PL_only'] = dict(); grid_drl_95c['PL_only'] = dict(); rates_weighted_exp['PL_only'] = dict(); rates_weighted_con['PL_only'] = dict(); grid_rates_95c['PL_only'] = dict()

# uninformative prior
bayesfactors['PL_only']['uninformative'] = dict(); best_params['PL_only']['uninformative'] = dict(); avg_params['PL_only']['uninformative'] = dict(); drl_weighted_avg['PL_only']['uninformative'] = dict(); posteriors_all['PL_only']['uninformative'] = dict(); posterior_95c['PL_only']['uninformative'] = dict(); grid_drl_95c['PL_only']['uninformative'] = dict(); rates_weighted_exp['PL_only']['uninformative'] = dict(); rates_weighted_con['PL_only']['uninformative'] = dict(); grid_rates_95c['PL_only']['uninformative'] = dict()
model_summary(grid_likelihood_xrange_KL, 'PL_only', 'uninformative', '4param', prior = None, params = ['exp_i', 'con_i', 'exp_p', 'con_p'], use_intercepts=False, grid_drl = inference_grid_final_xrange_norm_ps, rates = rates_all_xrange)

In [ ]:
# Load informative prior
prior_rates = pd.read_pickle('simulations/priors/prior_plonly.pickle')
prior_rates = prior_rates.reindex(grid_likelihood_xrange_KL.index)
prior_rates /= prior_rates.sum()

# informative prior
bayesfactors['PL_only']['informative'] = dict(); best_params['PL_only']['informative'] = dict(); avg_params['PL_only']['informative'] = dict(); drl_weighted_avg['PL_only']['informative'] = dict(); posteriors_all['PL_only']['informative'] = dict(); posterior_95c['PL_only']['informative'] = dict(); grid_drl_95c['PL_only']['informative'] = dict(); rates_weighted_exp['PL_only']['informative'] = dict(); rates_weighted_con['PL_only']['informative'] = dict(); grid_rates_95c['PL_only']['informative'] = dict()
model_summary(grid_likelihood_xrange_KL, 'PL_only', 'informative', '4param', prior = prior_rates, params = ['exp_i', 'con_i', 'exp_p', 'con_p'], use_intercepts=False, grid_drl = inference_grid_final_xrange_norm_ps, rates = rates_all_xrange)

In [ ]:
avg_params['PL_only']['uninformative'] = pd.concat(avg_params['PL_only']['uninformative']).unstack()
avg_params['PL_only']['informative'] = pd.concat(avg_params['PL_only']['informative']).unstack()
avg_params['PL_only'] = pd.concat(avg_params['PL_only'])

best_params['PL_only']['uninformative'] = pd.concat(best_params['PL_only']['uninformative']).unstack()
best_params['PL_only']['informative'] = pd.concat(best_params['PL_only']['informative']).unstack()
best_params['PL_only'] = pd.concat(best_params['PL_only'])

drl_weighted_avg['PL_only']['uninformative'] = pd.concat(drl_weighted_avg['PL_only']['uninformative']).unstack()
drl_weighted_avg['PL_only']['informative'] = pd.concat(drl_weighted_avg['PL_only']['informative']).unstack()
drl_weighted_avg['PL_only'] = pd.concat(drl_weighted_avg['PL_only'])

rates_weighted_exp['PL_only']['uninformative'] = pd.concat(rates_weighted_exp['PL_only']['uninformative']).unstack().T
rates_weighted_exp['PL_only']['informative'] = pd.concat(rates_weighted_exp['PL_only']['informative']).unstack().T
rates_weighted_exp['PL_only'] = pd.concat(rates_weighted_exp['PL_only'])

rates_weighted_con['PL_only']['uninformative'] = pd.concat(rates_weighted_con['PL_only']['uninformative']).unstack().T
rates_weighted_con['PL_only']['informative'] = pd.concat(rates_weighted_con['PL_only']['informative']).unstack().T
rates_weighted_con['PL_only'] = pd.concat(rates_weighted_con['PL_only'])

### Plot marginal posterior distributions

In [ ]:
plonly_lambda_list = list(range(5,21))
fig = make_subplots(rows = 1, cols = 2, shared_yaxes = True, horizontal_spacing = 0.08, subplot_titles = ['uninformative', 'informative'], x_title = 'onset length λ<sub>𝜀</sub> (nt)', y_title = 'onset length λ<sub>𝜅</sub> (nt)')

current_posterior = posteriors_all['PL_only']['uninformative']['4param'].set_index(['exp_i', 'con_i', 'exp_p', 'con_p'])[0]
current_posterior_marginal = pd.Series([current_posterior.reset_index().loc[(current_posterior.reset_index()['exp_i'] == exp) & (current_posterior.reset_index()['con_i'] == con)][0].sum() for exp in plonly_lambda_list for con in plonly_lambda_list], index = pd.MultiIndex.from_tuples([(exp, con)  for exp in plonly_lambda_list for con in plonly_lambda_list]))
current_posterior_marginal_cumulative = current_posterior_marginal.sort_values(ascending = False).cumsum()
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior_marginal_cumulative.where((current_posterior_marginal_cumulative > spacing[num]) & (current_posterior_marginal_cumulative <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['level_1', 'level_2']).reindex(current_posterior_marginal_cumulative.index)
fig.add_trace(go.Heatmap(x = posterior_contour.index.get_level_values(0), y = posterior_contour.index.get_level_values(1), z=((posterior_contour['level_0']+1).fillna(5)), coloraxis='coloraxis1'), row = 1, col = 1)

current_posterior = posteriors_all['PL_only']['informative']['4param'].set_index(['exp_i', 'con_i', 'exp_p', 'con_p'])[0]
current_posterior_marginal = pd.Series([current_posterior.reset_index().loc[(current_posterior.reset_index()['exp_i'] == exp) & (current_posterior.reset_index()['con_i'] == con)][0].sum() for exp in plonly_lambda_list for con in plonly_lambda_list], index = pd.MultiIndex.from_tuples([(exp, con)  for exp in plonly_lambda_list for con in plonly_lambda_list]))
current_posterior_marginal_cumulative = current_posterior_marginal.sort_values(ascending = False).cumsum()
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior_marginal_cumulative.where((current_posterior_marginal_cumulative > spacing[num]) & (current_posterior_marginal_cumulative <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['level_1', 'level_2']).reindex(current_posterior_marginal_cumulative.index)
fig.add_trace(go.Heatmap(x = posterior_contour.index.get_level_values(0), y = posterior_contour.index.get_level_values(1), z=((posterior_contour['level_0']+1).fillna(5)), coloraxis='coloraxis1'), row = 1, col = 2)

fig.update_layout(coloraxis1=dict(colorscale=discrete_deep_range, colorbar = dict(title = 'HDR', len=0.7, tickvals = [1.5,2.5,3.5,4.5], ticktext = ['68%', '95%', '99.7%', '99.99%']), cmin = 1, cmax = 5))
fig.update_yaxes(tickvals = plonly_lambda_list)
fig.update_xaxes(tickvals = plonly_lambda_list, scaleanchor="y", scaleratio=1)
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 320, width = 680, margin={'t':30,'l':60,'b':70,'r':10})
fig.show()

In [ ]:
fig.write_image('plots/fig_xrange_marginalposterior_uniform_restrictive.pdf')

### Plot HDR range and poster-weighted DRLs and rates

In [ ]:
current_model = 'PL_only'; current_prior = 'uninformative'; current_param = '4param'
current_posterior = posteriors_all['PL_only'][current_prior][current_param].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() <= spacing[num+1])).dropna(how = 'all')
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['exp_i', 'con_i', 'exp_p', 'con_p'])

In [ ]:
fig = make_fig_list_contour(inference_grid_final_xrange_norm_ps, posterior_contour, showgroups = [3,2,1,0], norm_range=(range(4,201)), xrange = True)
fig.add_trace(go.Scatter(x = list(L_range), y = (drl_weighted_avg['PL_only'].loc[current_prior].loc[current_param] / drl_weighted_avg['PL_only'].loc[current_prior].loc[current_param].sum())* bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL (uninformative)', line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[3], dash = 'dash', width = 4)))
fig.update_xaxes(range = [0, 2.33], tickvals = [1,2,5,10,20,50,100,200])

In [ ]:
fig.write_image('plots/DRL_CDFbands_PL_uniform_3mult_log.pdf')

In [ ]:
current_model = 'PL_only'; current_prior = 'informative'; current_param = '4param'
current_posterior = posteriors_all['PL_only'][current_prior][current_param].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() <= spacing[num+1])).dropna(how = 'all')
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['exp_i', 'con_i', 'exp_p', 'con_p'])

In [ ]:
fig = make_fig_list_contour(inference_grid_final_xrange_norm_ps, posterior_contour, showgroups = [3,2,1,0], norm_range=(range(4,201)), xrange = True)
fig.add_trace(go.Scatter(x = list(L_range), y = (drl_weighted_avg['PL_only'].loc[current_prior].loc[current_param] / drl_weighted_avg['PL_only'].loc[current_prior].loc[current_param].sum())* bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL (informative)', line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[3], dash = 'dash', width = 4)))
fig.update_xaxes(range = [0, 2.33], tickvals = [1,2,5,10,20,50,100,200])

In [ ]:
fig.write_image('plots/DRL_CDFbands_PL_informative_3mult_log.pdf')

In [ ]:
fig_rates = make_fig_rates_list_contour(None, rates_all_xrange)
for prior, dash in zip(['uninformative', 'informative'], [None, 'dot']):
    fig_rates.add_trace(go.Scatter(x = list(range(1,200)), y = rates_weighted_exp[current_model].loc[prior][current_param].reindex(range(1,200)), mode = 'lines', legendgroup = prior, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], dash = dash, width = 4), name = 'weighted 𝜀|<i>L</i> (' + prior + ')',))
    fig_rates.add_trace(go.Scatter(x = list(range(1,200)), y = rates_weighted_con[current_model].loc[prior][current_param].reindex(range(1,200)), mode = 'lines', legendgroup = prior, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[1], dash = dash, width = 4), name = 'weighted 𝜅|<i>L</i> (' + prior + ')',))
fig_rates.update_xaxes(range = [0, 2.33], tickvals = [1,2,5,10,20,50,100,200])
fig_rates.show()

In [ ]:
fig_rates.write_image('plots/rates_weighted_PL_priors_3mult_log.pdf')

### Onset length analysis

In [ ]:
xrange_cols_sorted = inference_grid_final_xrange.sort_index(axis=1).columns

In [ ]:
def mu_nu_comparision(param, threshold = 1.5, DRL_threshold = 1.5):
    diff_from_subonly = (inference_grid_final_xrange[param] * subonly_counts['A']['A'][1] / inference_grid_final_xrange[param][1]) / subonly_counts['A']['A'].dropna()
    L_diff_from_subonly = ((diff_from_subonly > threshold) | (diff_from_subonly < (1/threshold))).idxmax()
#    L_diff_from_subonly = ((diff_from_subonly > DRL_threshold)).idxmax()
    L_diff_from_subonly = np.nan if L_diff_from_subonly == 1 else L_diff_from_subonly
    exp_v_mu = ((rates_all_xrange.loc[param][1] * rates_all_xrange.loc[param][1].index) > threshold * 2*mu_nu_all['mu'][1]['A']).idxmax()
    con_v_mu = ((rates_all_xrange.loc[param][2] * rates_all_xrange.loc[param][2].index) > threshold * 2*mu_nu_all['mu'][1]['A']).idxmax()
    exp_v_nu = ((rates_all_xrange.loc[param][1] * rates_all_xrange.loc[param][1].index) > threshold * mu_nu_all['nu'][1]['A'] * rates_all_xrange.loc[param][1].index).idxmax()
    con_v_nu = ((rates_all_xrange.loc[param][2] * rates_all_xrange.loc[param][2].index) > threshold * mu_nu_all['nu'][1]['A'] * rates_all_xrange.loc[param][2].index).idxmax()

    con_exp_v_nu_mu = (((rates_all_xrange.loc[param][2] * rates_all_xrange.loc[param][2].index) - (rates_all_xrange.loc[param][1] * rates_all_xrange.loc[param][1].index)) > (mu_nu_all['nu'][1]['A'] * rates_all_xrange.loc[param][1].index - 2*mu_nu_all['mu'][1]['A'])).idxmax()


    max_con_exp_v_mu = ((pd.concat([(rates_all_xrange.loc[param][2] * rates_all_xrange.loc[param][2].index), (rates_all_xrange.loc[param][1] * rates_all_xrange.loc[param][1].index)], axis=1).max(axis=1)) > threshold * 2*mu_nu_all['mu'][1]['A']).idxmax()

    
    exp_v_mu_L = ((rates_all_xrange.loc[param][1] * rates_all_xrange.loc[param][1].index) > 0.1 * 2*mu_nu_all['mu'][1]['A'] / rates_all_xrange.loc[param][1].index).idxmax()

    if L_diff_from_subonly >1:
        exp_over_mu_at_L = (rates_all_xrange.loc[param][1][L_diff_from_subonly] * L_diff_from_subonly) /  (2*mu_nu_all['mu'][1]['A'])
        exp_over_muexp_at_L = (rates_all_xrange.loc[param][1][L_diff_from_subonly] * L_diff_from_subonly)  /  ((rates_all_xrange.loc[param][1][L_diff_from_subonly] * L_diff_from_subonly) + (2*mu_nu_all['mu'][1]['A']))
    else:
        exp_over_mu_at_L = np.nan; exp_over_muexp_at_L = np.nan
    
    return (L_diff_from_subonly, exp_v_mu, con_v_mu, exp_v_nu, con_v_nu, con_exp_v_nu_mu, exp_v_mu_L, exp_over_mu_at_L, exp_over_muexp_at_L, max_con_exp_v_mu)

In [ ]:
threshold = 2
mu_comparison = pd.Series([(((denovo_exp_rate[motif] * denovo_exp_rate[motif].index) / (2*mu_nu_all['mu'][len(motif)][motif])) > threshold).idxmax() for motif in subonly_counts['A']], index = subonly_counts['A'].columns)
mu_comparison_decode = pd.Series([(((decode_exp_rate[motif] * decode_exp_rate[motif].index) / (2*mu_nu_all['mu'][len(motif)][motif])) > threshold).idxmax() for motif in subonly_counts['A']], index = subonly_counts['A'].columns)
subonly_comparison = pd.Series([(((((bootstrap_counts_mean[motif] * (subonly_counts['A'][motif][1] / bootstrap_counts_mean[motif][1])) / subonly_counts['A'][motif]).dropna()) > threshold) | ((((bootstrap_counts_mean[motif] * (subonly_counts['A'][motif][1] / bootstrap_counts_mean[motif][1])) / subonly_counts['A'][motif]).dropna()) < 1/threshold)).idxmax() for motif in subonly_counts['A']], index = subonly_counts['A'].columns)
mu_expcon_comparison = pd.Series([((pd.concat([(denovo_exp_rate[motif] * denovo_exp_rate[motif].index), (denovo_con_rate[motif] * denovo_con_rate[motif].index)], axis=1).max(axis=1) > (threshold * 2*mu_nu_all['mu'][len(motif)][motif])) ).idxmax() for motif in subonly_counts['A']], index = subonly_counts['A'].columns)

mu_comparison_empirical = pd.concat([mu_comparison, mu_comparison_decode, subonly_comparison, mu_expcon_comparison], axis=1)
mu_comparison_empirical.columns = ['mu_L', 'mu_L_decode', 'DRL_L', 'mu_expcon_max']
mu_comparison_empirical['exp_over_mu_at_L'] = [(denovo_exp_rate[motif].replace(0, np.nan)[DRL_L] * DRL_L) / (2*mu_nu_all['mu'][len(motif)][motif]) for motif, DRL_L in zip(mu_comparison_empirical['DRL_L'].index, mu_comparison_empirical['DRL_L'])]
mu_comparison_empirical['exp_over_mu_at_L_decode'] = [(decode_exp_rate[motif][DRL_L] * DRL_L) / (2*mu_nu_all['mu'][len(motif)][motif]) for motif, DRL_L in zip(mu_comparison_empirical['DRL_L'].index, mu_comparison_empirical['DRL_L'])]
mu_comparison_empirical['exp_over_muexp_at_L'] = [(denovo_exp_rate[motif].replace(0, np.nan)[DRL_L] * DRL_L) / ((denovo_exp_rate[motif].replace(0, np.nan)[DRL_L] * DRL_L) + (2*mu_nu_all['mu'][len(motif)][motif])) for motif, DRL_L in zip(mu_comparison_empirical['DRL_L'].index, mu_comparison_empirical['DRL_L'])]
mu_comparison_empirical['exp_over_muexp_at_L_decode'] = [(decode_exp_rate[motif].replace(0, np.nan)[DRL_L] * DRL_L) / ((decode_exp_rate[motif].replace(0, np.nan)[DRL_L] * DRL_L) + (2*mu_nu_all['mu'][len(motif)][motif])) for motif, DRL_L in zip(mu_comparison_empirical['DRL_L'].index, mu_comparison_empirical['DRL_L'])]

for lam in ['mu_L', 'mu_L_decode', 'DRL_L', 'mu_expcon_max']:
    mu_comparison_empirical[lam] *= mu_comparison_empirical.index.str.len()

In [ ]:
fig_mu_nu_comparison = go.Figure()
fig_mu_nu_comparison.add_trace(go.Scatter(x = mu_comparison_empirical['DRL_L'], y = mu_comparison_empirical['mu_expcon_max'], name = 'max_expcon_v_mu_empirical', mode = 'markers', showlegend = False, text = list(mu_comparison_empirical.index), marker = dict(size = 30, opacity = 0.3)))
fig_mu_nu_comparison.add_trace(go.Scatter(x = [0,30], y = [0,30], mode = 'lines', line = dict(color = 'black', width = 2), opacity = 0.3, showlegend = False))

for motif in mu_comparison_empirical.index:
    fig_mu_nu_comparison.add_annotation(x=np.log10(mu_comparison_empirical['DRL_L'][motif]), y=np.log10(mu_comparison_empirical['mu_expcon_max'][motif]), 
            text=motif,
            font=dict(color="black", size=12),
            ax = 0,
            ay = -6 if motif in ['AG','AAG','ACC','AAC'] else 6 if motif in ['AT','AGG','ATC','ACG'] else 0,
            showarrow=True,
            arrowcolor = 'rgba(255,0,0,0)')

fig_mu_nu_comparison.update_yaxes(type = 'log', range = [0,1.333], tickvals = [1,3,6,9,12,15], title = 'onset length (nt) from rate estimates')
fig_mu_nu_comparison.update_xaxes(type = 'log', range = [0,1.333], tickvals = [1,3,6,9,12,15], title = 'onset length (nt) from DRLs', scaleanchor="y", scaleratio=1)
fig_mu_nu_comparison.update_layout(width = 600, height = 600,  font=dict(family = 'Arial', size = 20), margin={'t':20,'l':100,'b':60,'r':10})
fig_mu_nu_comparison.show()

In [ ]:
fig_mu_nu_comparison.write_image('plots/lambda_comparison.pdf')

## Logarithmic power parameterization

In [ ]:
# Normalize counts (with pseudocount)
inference_grid_final_log_norm_ps = (inference_grid_final_log.reindex(norm_range) + pseudocount) / (inference_grid_final_log.reindex(norm_range) + pseudocount).sum()
# Calculate KL
KL_inference_grid_final_log = inference_grid_final_log_norm_ps.apply(lambda y: calc_KL_divergence_CHM13(y, L_range=L_range))
# Calculate acceptance probability
grid_likelihood_log_KL = calc_likelihood_from_primate_metric_KL(KL_inference_grid_final_log)

In [ ]:
grid_likelihood = grid_likelihood_log_KL.copy()

#### grid of rates

In [ ]:
def log_curve_from_L9(power, intercept, bins = 200):
    return pd.Series([intercept_list[intercept] * (np.log(1+L-8) / np.log(1+9 -8))**power for L in range(9,bins+1)], index = range(9,bins+1))
rates_all_log = pd.DataFrame([((exp_i, con_i, exp_p, con_p,), pd.concat([denovo_exp_rate['A'].loc[:8], log_curve_from_L9(exp_p, exp_i, 200)]), pd.concat([denovo_con_rate['A'].loc[:8], log_curve_from_L9(con_p, con_i, 200)])) for (exp_i, con_i, exp_p, con_p,) in inference_grid_final_log.columns], index = inference_grid_final_log.columns)

### Calculate Bayes factors, etc.

In [ ]:
bayesfactors['log'] = dict(); best_params['log'] = dict(); avg_params['log'] = dict(); drl_weighted_avg['log'] = dict(); posteriors_all['log'] = dict(); posterior_95c['log'] = dict(); grid_drl_95c['log'] = dict(); rates_weighted_exp['log'] = dict(); rates_weighted_con['log'] = dict(); grid_rates_95c['log'] = dict()

In [ ]:
# uniform prior
bayesfactors['log']['uniform'] = dict(); best_params['log']['uniform'] = dict(); avg_params['log']['uniform'] = dict(); drl_weighted_avg['log']['uniform'] = dict(); posteriors_all['log']['uniform'] = dict(); posterior_95c['log']['uniform'] = dict(); grid_drl_95c['log']['uniform'] = dict(); rates_weighted_exp['log']['uniform'] = dict(); rates_weighted_con['log']['uniform'] = dict(); grid_rates_95c['log']['uniform'] = dict()

model_summary(grid_likelihood_log_KL, 'log', 'uniform', '4param', prior = None, params = ['exp_i', 'con_i', 'exp_p', 'con_p'], grid_drl = inference_grid_final_log_norm_ps, rates = rates_all_log)

# 2 parameter Bayes factor, model where power law is same, intercept is same
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_p'] == partial_grid['con_p']) & (partial_grid['exp_i'] == partial_grid['con_i'])).dropna()
model_summary(partial_grid, 'log', 'uniform', '2param', prior = None, params = ['exp_i', 'exp_p'], grid_drl = inference_grid_final_log_norm_ps, rates = rates_all_log)

# Multiplier 3 parameter model
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_i'] == 1+ partial_grid['con_i'])).dropna()
model_summary(partial_grid, 'log', 'uniform', '3param_Mult', prior = None, params = ['M', 'exp_p', 'con_p'], grid_drl = inference_grid_final_log_norm_ps, rates = rates_all_log)

In [ ]:
# restrictive prior
prior_rates = pd.read_pickle('simulations/grid_4param_v2/prior_log_restrictive.pickle')
prior_rates = prior_rates.reindex(grid_likelihood.index)
prior_rates /= prior_rates.sum()

# restrictive prior
bayesfactors['log']['restrictive'] = dict(); best_params['log']['restrictive'] = dict(); avg_params['log']['restrictive'] = dict(); drl_weighted_avg['log']['restrictive'] = dict(); posteriors_all['log']['restrictive'] = dict(); posterior_95c['log']['restrictive'] = dict(); grid_drl_95c['log']['restrictive'] = dict(); rates_weighted_exp['log']['restrictive'] = dict(); rates_weighted_con['log']['restrictive'] = dict(); grid_rates_95c['log']['restrictive'] = dict()

model_summary(grid_likelihood, 'log', 'restrictive', '4param', prior = prior_rates, params = ['exp_i', 'con_i', 'exp_p', 'con_p'], grid_drl = inference_grid_final_log_norm_ps, rates = rates_all_log)

# 2 parameter Bayes factor, model where power law is same, intercept is same
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_p'] == partial_grid['con_p']) & (partial_grid['exp_i'] == partial_grid['con_i'])).dropna()
model_summary(partial_grid, 'log', 'restrictive', '2param', prior = prior_rates, params = ['exp_i', 'exp_p'], grid_drl = inference_grid_final_log_norm_ps, rates = rates_all_log)


# Multiplier 3 parameter model
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_i'] == 1+ partial_grid['con_i'])).dropna()
model_summary(partial_grid, 'log', 'restrictive', '3param_Mult', prior = prior_rates, params = ['M', 'exp_p', 'con_p'], grid_drl = inference_grid_final_log_norm_ps, rates = rates_all_log)

In [ ]:
# permissive prior
prior_rates = pd.read_pickle('simulations/grid_4param_v2/prior_log_relaxed.pickle')
prior_rates = prior_rates.reindex(grid_likelihood.index)
prior_rates /= prior_rates.sum()

# permissive prior
bayesfactors['log']['permissive'] = dict(); best_params['log']['permissive'] = dict(); avg_params['log']['permissive'] = dict(); drl_weighted_avg['log']['permissive'] = dict(); posteriors_all['log']['permissive'] = dict(); posterior_95c['log']['permissive'] = dict(); grid_drl_95c['log']['permissive'] = dict(); rates_weighted_exp['log']['permissive'] = dict(); rates_weighted_con['log']['permissive'] = dict(); grid_rates_95c['log']['permissive'] = dict()
model_summary(grid_likelihood, 'log', 'permissive', '4param', prior = prior_rates, params = ['exp_i', 'con_i', 'exp_p', 'con_p'], grid_drl = inference_grid_final_log_norm_ps, rates = rates_all_log)

# 2 parameter Bayes factor, model where power law is same, intercept is same
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_p'] == partial_grid['con_p']) & (partial_grid['exp_i'] == partial_grid['con_i'])).dropna()
model_summary(partial_grid, 'log', 'permissive', '2param', prior = prior_rates, params = ['exp_i', 'exp_p'], grid_drl = inference_grid_final_log_norm_ps, rates = rates_all_log)

# Multiplier 3 parameter model
partial_grid = grid_likelihood.copy().reset_index()
partial_grid = partial_grid.where((partial_grid['exp_i'] == 1+ partial_grid['con_i'])).dropna()
model_summary(partial_grid, 'log', 'permissive', '3param_Mult', prior = prior_rates, params = ['M', 'exp_p', 'con_p'], grid_drl = inference_grid_final_log_norm_ps, rates = rates_all_log)

In [ ]:
avg_params['log']['uniform'] = pd.concat(avg_params['log']['uniform']).unstack()
avg_params['log']['restrictive'] = pd.concat(avg_params['log']['restrictive']).unstack()
avg_params['log']['permissive'] = pd.concat(avg_params['log']['permissive']).unstack()
avg_params['log'] = pd.concat(avg_params['log'])

best_params['log']['uniform'] = pd.concat(best_params['log']['uniform']).unstack()
best_params['log']['restrictive'] = pd.concat(best_params['log']['restrictive']).unstack()
best_params['log']['permissive'] = pd.concat(best_params['log']['permissive']).unstack()
best_params['log'] = pd.concat(best_params['log'])

drl_weighted_avg['log']['uniform'] = pd.concat(drl_weighted_avg['log']['uniform']).unstack()
drl_weighted_avg['log']['restrictive'] = pd.concat(drl_weighted_avg['log']['restrictive']).unstack()
drl_weighted_avg['log']['permissive'] = pd.concat(drl_weighted_avg['log']['permissive']).unstack()
drl_weighted_avg['log'] = pd.concat(drl_weighted_avg['log'])

rates_weighted_exp['log']['uniform'] = pd.concat(rates_weighted_exp['log']['uniform']).unstack().T
rates_weighted_exp['log']['restrictive'] = pd.concat(rates_weighted_exp['log']['restrictive']).unstack().T
rates_weighted_exp['log']['permissive'] = pd.concat(rates_weighted_exp['log']['permissive']).unstack().T
rates_weighted_exp['log'] = pd.concat(rates_weighted_exp['log'])

rates_weighted_con['log']['uniform'] = pd.concat(rates_weighted_con['log']['uniform']).unstack().T
rates_weighted_con['log']['restrictive'] = pd.concat(rates_weighted_con['log']['restrictive']).unstack().T
rates_weighted_con['log']['permissive'] = pd.concat(rates_weighted_con['log']['permissive']).unstack().T
rates_weighted_con['log'] = pd.concat(rates_weighted_con['log'])

### Plot posterior distributions

In [ ]:
fig = make_subplots(rows = 3, cols = 7, shared_xaxes = True, shared_yaxes = True, horizontal_spacing = 0.02, vertical_spacing = 0.08, subplot_titles = ['<i>m</i>=' + str(round(x / intercept_list[1], 1)) for x in intercept_list[1:]], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')

current_posterior = posteriors_all['log']['uniform']['3param_Mult'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
row_count=1; col_count = 0
for m in m_list:
    col_count+=1
    fig.add_trace(go.Heatmap(x = posterior_contour.loc[m].index.get_level_values(0), y = posterior_contour.loc[m].index.get_level_values(1), z=(posterior_contour.loc[m]['level_0']+1), coloraxis='coloraxis1'), row = row_count, col = col_count)
# best parameter combination
fig.add_annotation(x=current_posterior.iloc[0]['exp_p'], y=current_posterior.iloc[0]['con_p'], showarrow=True, arrowhead=1, arrowcolor = 'red', row = 1, col = pd.Series(range(1,8), index = m_list).loc[current_posterior.iloc[0]['M']])

current_posterior = posteriors_all['log']['permissive']['3param_Mult'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
row_count=2; col_count = 0
for m in m_list:
    col_count+=1
    fig.add_trace(go.Heatmap(x = posterior_contour.loc[m].index.get_level_values(0), y = posterior_contour.loc[m].index.get_level_values(1), z=(posterior_contour.loc[m]['level_0']+1), coloraxis='coloraxis1'), row = row_count, col = col_count)
# best parameter combination
fig.add_annotation(x=current_posterior.iloc[0]['exp_p'], y=current_posterior.iloc[0]['con_p'], showarrow=True, arrowhead=1, arrowcolor = 'red', row = 2, col = pd.Series(range(1,8), index = m_list).loc[current_posterior.iloc[0]['M']])

current_posterior = posteriors_all['log']['restrictive']['3param_Mult'].sort_values(by = [0], ascending = False)
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])
row_count=3; col_count = 0
for m in m_list:
    col_count+=1
    fig.add_trace(go.Heatmap(x = posterior_contour.loc[m].index.get_level_values(0), y = posterior_contour.loc[m].index.get_level_values(1), z=(posterior_contour.loc[m]['level_0']+1), coloraxis='coloraxis1'), row = row_count, col = col_count)
# best parameter combination
fig.add_annotation(x=current_posterior.iloc[0]['exp_p'], y=current_posterior.iloc[0]['con_p'], showarrow=True, arrowhead=1, arrowcolor = 'red', row = 3, col = pd.Series(range(1,8), index = m_list).loc[current_posterior.iloc[0]['M']])

fig.update_layout(coloraxis1=dict(colorscale=discrete_deep_range, colorbar = dict(title = 'HDR', len=0.55, tickvals = [1.5,2.5,3.5,4.5], ticktext = ['68%', '95%', '99.7%', '99.99%'], x = 1.05), cmin = 1, cmax = 5))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 420, width = 1000, margin={'t':50,'l':60,'b':55,'r':10})
fig.update_yaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)

fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)

fig.show()

In [ ]:
fig.write_image('plots/posterior_dist_contour_PL_3mult_uniform_permissive_restrictive_log.pdf')

### Plot HDR range and posterior-weighted DRLs and rates

In [ ]:
current_model = 'log'; current_prior = 'uniform'; current_param = '3param_Mult'
current_posterior = posteriors_all[current_model][current_prior][current_param].sort_values(by = [0], ascending = False)

In [ ]:
posterior_contour = dict()
for num in range(len(spacing)-1):
    posterior_contour[num] = current_posterior.where((current_posterior[0].cumsum() > spacing[num]) & (current_posterior[0].cumsum() <= spacing[num+1])).dropna()
posterior_contour = pd.concat(posterior_contour).reset_index().set_index(['M', 'exp_p', 'con_p'])

In [ ]:
fig = make_fig_list_contour(inference_grid_final_log_norm_ps, posterior_contour, norm_range=(range(4,201)), showgroups=[1], tracename = '95% HDR (uniform prior)')
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg[current_model].loc[current_prior].loc[current_param] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL (uniform)', line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], width = 3)))
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg[current_model].T['permissive'][current_param] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL (permissive)', line = dict(width = 3, dash = 'dash', color = plotly.colors.DEFAULT_PLOTLY_COLORS[0])))
fig.add_trace(go.Scatter(x = list(L_range), y = drl_weighted_avg[current_model].T['restrictive'][current_param] * bootstrap_counts_mean['A'].reindex(L_range).sum(), name = 'weighted DRL (restrictive)', line = dict(width = 3, dash = 'dot', color = plotly.colors.DEFAULT_PLOTLY_COLORS[0])))

In [ ]:
fig.write_image('plots/DRL_95HDR_log_uniform_3mult.pdf')

In [ ]:
fig_rates = make_fig_rates_list_contour(None, rates_all)
for prior, dash in zip(['uniform', 'permissive', 'restrictive'], [None, 'dash', 'dot']):
    fig_rates.add_trace(go.Scatter(x = list(range(9,200)), y = rates_weighted_exp[current_model].loc[prior][current_param].reindex(range(9,200)), mode = 'lines', legendgroup = prior, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[0], dash = dash, width = 4), name = 'weighted 𝜀|<i>L</i> (' + prior + ')',))
    fig_rates.add_trace(go.Scatter(x = list(range(9,200)), y = rates_weighted_con[current_model].loc[prior][current_param].reindex(range(9,200)), mode = 'lines', legendgroup = prior, line = dict(color = plotly.colors.DEFAULT_PLOTLY_COLORS[1], dash = dash, width = 4), name = 'weighted 𝜅|<i>L</i> (' + prior + ')',))
fig_rates.show()

In [ ]:
fig_rates.write_image('plots/rates_95c_weighted_log_allpriors_3mult.pdf')

# Collection of info for Table 1

In [ ]:
bayesfactors_summary = pd.DataFrame.from_dict(bayesfactors).T.stack().apply(pd.Series)

In [ ]:
bayesfactors_summary

In [ ]:
bayesfactors_summary.div(bayesfactors_summary['2param'], axis=0)

In [ ]:
pd.concat(best_params)

In [ ]:
pd.concat(avg_params)

# Genome size plot

In [ ]:
genome_size_final = inference_grid_final_nointerp_subonly.mul(inference_grid_final_nointerp_subonly.index, axis=0).sum()# + inference_grid_final_nointerp_subonly_Bdist.mul(inference_grid_final_nointerp_subonly_Bdist.index, axis=0).sum()
genome_size_final = genome_size_final.reset_index()
genome_size_final['M'] = pd.Series(intercept_list / intercept_list[1]).reindex(genome_size_final['exp_i']).values
genome_size_final = genome_size_final.set_index(['M', 'exp_p', 'con_p'])[0]

In [ ]:
# Fig. S12a
fig = make_subplots(rows = 1, cols = 7, shared_xaxes = True, shared_yaxes = True, horizontal_spacing = 0.01, subplot_titles = ['<i>m</i>=' + str(round(x / intercept_list[1], 1)) for x in intercept_list[1:]], x_title = 'expansion power 𝜏<sub>𝜀</sub>', y_title = 'contraction power 𝜏<sub>𝜅</sub>')
row_count=1; col_count = 0
for m in m_list:
    col_count+=1
    fig.add_trace(go.Heatmap(x = genome_size_final.loc[m].index.get_level_values(0), y = genome_size_final.loc[m].index.get_level_values(1), z=np.log10(genome_size_final.loc[m].replace(0, 1e50)), coloraxis='coloraxis1'), row = row_count, col = col_count)
fig.update_layout(coloraxis1=dict(colorscale='deep_R', colorbar = dict(title = 'log<sub>10</sub>(total nt)', len=1), cmin = 9.3, cmax= 50))
fig.update_layout(font=dict(family = 'Arial', size = 16), height = 200, width = 1000, margin={'t':30,'l':50,'b':55,'r':10})
fig.update_yaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)')
fig.update_xaxes(range = [0,4.2], dtick = 1, gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="y", scaleratio=1)

fig.update_annotations(selector={"text":"'expansion power 𝜏<sub>𝜀</sub>'"}, yshift=-30, x=0.5)
fig.update_annotations(selector={"text":"contraction power 𝜏<sub>𝜅</sub>"}, xshift=-30, y=0.5)
fig.show()

In [ ]:
fig.write_image('plots/fig_S12a.png', scale=10)
fig.write_image('plots/fig_S12a.pdf')

# Plot DRLs according to delta-tau values

In [ ]:
grid_index = inference_grid_final_nointerp_subonly.T.reset_index()[['exp_i', 'con_i', 'exp_p', 'con_p']]

# ignore lower left corner
exp_i = 3; con_i = 2
corner_size = 1
diagonal_groups = dict()
for diag in [0, -0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9, -1, -1.1, -1.2, -1.3, -1.4, -1.5, -1.6, -1.7, -1.8, -1.9, -2, -2.1, -2.2, -2.3, -2.4, -2.5, -2.6, -2.7, -2.8, -2.9, -3, -3.1, -3.2, -3.3, -3.4, -3.5, -3.6, -3.7, -3.8, -3.9, -4]:
    diagonal_groups[diag] = grid_index.loc[(grid_index['exp_i'] == exp_i) & (grid_index['con_i'] == con_i) & (grid_index['exp_p'] - grid_index['con_p'] >= diag-0.01) & (grid_index['exp_p'] - grid_index['con_p'] <= diag+0.01)].copy()#  & ((grid_index['exp_p'] >corner_size) | (grid_index['con_p'] >corner_size))].copy()
    if len(diagonal_groups[diag]) == 0:
        del diagonal_groups[diag]

diagonal_groups_exp = dict()
for diag in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4]:
    diagonal_groups_exp[diag] = grid_index.loc[(grid_index['exp_i'] == exp_i) & (grid_index['con_i'] == con_i) & (grid_index['exp_p'] - grid_index['con_p'] >= diag-0.01) & (grid_index['exp_p'] - grid_index['con_p'] <= diag+0.01)].copy()#  & ((grid_index['exp_p'] >corner_size) | (grid_index['con_p'] >corner_size))].copy()
    if len(diagonal_groups_exp[diag]) == 0:
        del diagonal_groups_exp[diag]

In [ ]:
color_adjustment = 5
current_colorscale = plotly.colors.sample_colorscale(plotly.colors.sequential.Rainbow, np.linspace(0,(len(diagonal_groups.keys())+color_adjustment) / (len(diagonal_groups.keys())+color_adjustment), len(diagonal_groups.keys())+color_adjustment))

In [ ]:
# Fig. 6a (illustration of dtau in parameter space)
fig_phases = go.Figure()
for diag in list(diagonal_groups_exp.keys())[:-1]:
    fig_phases.add_trace(go.Scatter(x = diagonal_groups_exp[diag]['exp_p'], y = diagonal_groups_exp[diag]['con_p'], name = str(diag), mode = 'markers', opacity = 0.75, showlegend = False, marker = dict(symbol = 'square', size = 6, color = 'rgba(120,120,120,0.65)')))
colornum = 0
for diag in diagonal_groups:
    fig_phases.add_trace(go.Scatter(x = diagonal_groups[diag]['exp_p'], y = diagonal_groups[diag]['con_p'], name = str(diag), mode = 'markers', opacity = 0.75, showlegend = False, marker = dict(symbol = 'square', size = 6, color = current_colorscale[colornum])))
    colornum +=1

fig_phases.add_trace(go.Scatter(x = [0], y = [3.5], marker = dict(color = 'red', size = 8, line_width = 1, line_color = 'white', symbol = 'circle'), mode = 'markers', name = 'i'))
fig_phases.add_trace(go.Scatter(x = [1], y = [2.5], marker = dict(color = 'red', size = 9, line_width = 1, line_color = 'white', symbol = 'diamond'), mode = 'markers', name = 'ii'))
fig_phases.add_trace(go.Scatter(x = [1.5], y = [2], marker = dict(color = 'red', size = 8, line_width = 1, line_color = 'white', symbol = 'square'), mode = 'markers', name = 'iii'))
fig_phases.add_trace(go.Scatter(x = [1.7], y = [1.8], marker = dict(color = 'red', size = 11, line_width = 1, line_color = 'white', symbol = 'triangle-up'), mode = 'markers', name = 'iv'))
fig_phases.add_trace(go.Scatter(x = [2.5], y = [1], marker = dict(color = 'red', size = 11, line_width = 1, line_color = 'white', symbol = 'triangle-down'), mode = 'markers', name = 'v'))

fig_phases.update_layout(font=dict(family = 'Arial', size = 14), height = 250, width = 270, margin={'t':30,'l':35,'b':40,'r':0})
fig_phases.update_xaxes(range = [-0.2, 3.5], dtick = 1, title = 'expansion power 𝜏<sub>𝜀</sub>', gridcolor = 'rgba(0,0,0,0.2)')
fig_phases.update_yaxes(range = [0, 3.58], dtick = 1, title = 'contraction power 𝜏<sub>𝜅</sub>', gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="x", scaleratio=1)
fig_phases.update_layout(legend=dict(indentation = -15))
fig_phases.show()

In [ ]:
fig_phases.write_image('plots/fig6a.svg')
fig_phases.write_image('plots/fig6a.pdf')

#### Plots for Fig. S11


In [ ]:
def fig_add_paramlist(fig, param_list, group_name, colorlist = plotly.colors.cyclical.Phase, motif = 'A', colornum = 0, df = inference_grid_final_nointerp_subonly, opacity = 0.5, show_CHM13 = False):
    group = df.reindex(param_list.set_index(['exp_i', 'con_i', 'exp_p', 'con_p']).index, axis=1).copy()
    group.columns = list(range(len(group.columns)))
    group = group[group >1].fillna(1,limit=1, axis=0)
    group.loc[len(group)+1] = np.nan
    
    # remove last n bins (if part of reflective boundary)
    for i in range(8):
        group_last = group[group >1].isna().idxmax().replace(1,200) -1; group_cols = group.columns
        group = group.unstack()
        group.loc[zip(group_cols, group_last)] = [np.nan if val > 1e2 else val for val in group.loc[zip(group_cols, group_last)]]
        group = group.unstack().T
    group_index = np.array(([group.index]*200)).flatten()
    # normalize
    group = group / group.sum()
    group = pd.Series(group.transpose().to_numpy().flatten())

    fig.add_trace(go.Scatter(x = group_index, y = group, mode = 'lines', line = dict(width = 1.5, color = colorlist[colornum] if colornum != -1 else 'rgba(120,120,120,0.2)'), name = group_name, connectgaps = False, opacity = opacity))
    if show_CHM13 == True:
            fig.add_trace(go.Scatter(x = bootstrap_counts_mean[motif].index, y = bootstrap_counts_mean[motif] / bootstrap_counts_mean[motif].sum(), line = dict(color = 'rgba(0,0,0,1)', width = 3), legendgroup = 'ci', name = 'T2T-CHM13'))
            fig.add_trace(go.Scatter(x = bootstrap_counts_max[motif].index, y = bootstrap_counts_max[motif] / bootstrap_counts_mean[motif].sum(), line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 95%'))
            fig.add_trace(go.Scatter(x = bootstrap_counts_min[motif].index, y = bootstrap_counts_min[motif] / bootstrap_counts_mean[motif].sum(), fill='tonexty', fillcolor = 'rgba(0,0,0,0.25)', line = dict(color = 'rgba(0,0,0,0)'), legendgroup = 'ci', showlegend = False, name = 'CHM13 bootstrap 5%'))
    fig.update_xaxes(type = 'log', title = 'repeat tract length (units)', range = [0,2.5], gridcolor = 'rgba(0,0,0,0.2)')
    fig.update_yaxes(type = 'log', title = 'counts (normalized)', range = [-9.1,0], gridcolor = 'rgba(0,0,0,0.2)', tickformat = '1.0e', dtick = 2)
    fig.update_layout(font=dict(family = 'Arial', size = 16), height = 300, width = 780, margin={'t':20,'l':60,'b':40,'r':10}) 

In [ ]:
# ignore lower left corner
for exp_i, con_i, corner_size in zip(range(1,8), range(7), [1.8, 1.6, 1.4, 1.2, 1, 0.8, 0.6]):
    diagonal_groups = dict()
    for diag in [0, -0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9, -1, -1.1, -1.2, -1.3, -1.4, -1.5, -1.6, -1.7, -1.8, -1.9, -2, -2.1, -2.2, -2.3, -2.4, -2.5, -2.6, -2.7, -2.8, -2.9, -3, -3.1, -3.2, -3.3, -3.4, -3.5, -3.6, -3.7, -3.8, -3.9, -4]:
        diagonal_groups[diag] = grid_index.loc[(grid_index['exp_i'] == exp_i) & (grid_index['con_i'] == con_i) & (grid_index['exp_p'] - grid_index['con_p'] >= diag-0.01) & (grid_index['exp_p'] - grid_index['con_p'] <= diag+0.01)  & ((grid_index['exp_p'] >corner_size) | (grid_index['con_p'] >corner_size))].copy()
        if len(diagonal_groups[diag]) == 0:
            del diagonal_groups[diag]
    
    diagonal_groups_exp = dict()
    for diag in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4]:
        diagonal_groups_exp[diag] = grid_index.loc[(grid_index['exp_i'] == exp_i) & (grid_index['con_i'] == con_i) & (grid_index['exp_p'] - grid_index['con_p'] >= diag-0.01) & (grid_index['exp_p'] - grid_index['con_p'] <= diag+0.01)  & ((grid_index['exp_p'] >corner_size) | (grid_index['con_p'] >corner_size))].copy()
        if len(diagonal_groups_exp[diag]) == 0:
            del diagonal_groups_exp[diag]

    fig_phases = go.Figure()
    for diag in list(diagonal_groups_exp.keys())[:-1]:
        fig_phases.add_trace(go.Scatter(x = diagonal_groups_exp[diag]['exp_p'], y = diagonal_groups_exp[diag]['con_p'], name = str(diag), mode = 'markers', opacity = 0.75, showlegend = False, marker = dict(symbol = 'square', size = 4.5, color = 'rgba(120,120,120,0.65)')))
    colornum = 0
    for diag in diagonal_groups:
        fig_phases.add_trace(go.Scatter(x = diagonal_groups[diag]['exp_p'], y = diagonal_groups[diag]['con_p'], name = str(diag), mode = 'markers', opacity = 0.75, showlegend = False, marker = dict(symbol = 'square', size = 4.5, color = current_colorscale[colornum])))
        colornum +=1
    fig_phases.update_layout(title = '<i>m</i>='+str(round(intercept_list[exp_i] / intercept_list[1], 2)), font=dict(family = 'Arial', size = 12), height = 220, width = 220, margin={'t':30,'l':35,'b':40,'r':10})
    fig_phases.update_xaxes(range = [0,4.1], dtick = 1, title = 'expansion power 𝜏<sub>𝜀</sub>', gridcolor = 'rgba(0,0,0,0.2)')
    fig_phases.update_yaxes(range = [0,4.1], dtick = 1, title = 'contraction power 𝜏<sub>𝜅</sub>', gridcolor = 'rgba(0,0,0,0.2)', scaleanchor="x", scaleratio=1)
    

    fig = go.Figure()
    for diag in list(diagonal_groups_exp.keys())[:-1]:
        fig_add_paramlist(fig, diagonal_groups_exp[diag], group_name = str(diag), colornum = -1, show_CHM13 = False)
    colornum = 0
    for diag in list(diagonal_groups.keys())[:-1]:
        fig_add_paramlist(fig, diagonal_groups[diag], group_name = str(diag), colorlist = current_colorscale, colornum = colornum, opacity = 0.4, show_CHM13 = False)
        colornum +=1
    for diag in list(diagonal_groups.keys())[-1:]:
        fig_add_paramlist(fig, diagonal_groups[diag], group_name = str(diag), colorlist = current_colorscale, colornum = colornum, opacity = 0.4, show_CHM13 = True)
    fig.update_xaxes(range=[0,2.33], tickvals = [1,2,5,10,20,50,100,200])
    fig.update_layout(font=dict(family = 'Arial', size = 14), height = 250, width = 520, margin={'t':30,'l':60,'b':40,'r':10}, showlegend = False)
    
    fig_phases.write_image('plots/figS5a_mult'+str(exp_i)+'_' + str(con_i)+'.pdf')
    fig.write_image('plots/figS5b_mult'+str(exp_i)+'_' + str(con_i)+'.pdf')

In [ ]:
# Show example
fig_phases.show()
fig.show()

# Flux plots for analytics comparison

#### Plot functions

In [ ]:
# evolve function
def mut_evolve_dist_AB(A_count_input, B_count_input, starting_conditions, boot = None, input_nuc = 'A', mut = True, mutonly = False, speedup_multiplier = 1, output_components = False, stochastics = None, reflective = True, boundary_count = 1000):
    exp_rate_A_AA, con_rate_A_AA, nonexp_rate_A_AB, B_indel_rates = starting_conditions
    A_count_output = A_count_input.copy(); B_count_output = B_count_input.copy()
    A_bins = len(A_count_input)
    B_bins = len(B_count_input)
    A_length_array = np.array(range(1,A_bins+3))
    A_length_array_bases = np.array(range(1,A_bins+3)) * len(input_nuc) ### including motif length
    B_length_array = np.array(range(1,B_bins+3))
    B_length_array_bases = np.array(range(1,B_bins+3)) * len(input_nuc) ### including motif length
    A_count_input = np.insert(A_count_input, A_bins, [0,0])
    B_count_input = np.insert(B_count_input, B_bins, [0,0])
    if boot is None:
        denovo_sub = denovo_substitution_context_rate.loc[input_nuc]
    else:
        denovo_sub = denovo_substitution_context_rate_poisson.loc[boot].loc[input_nuc]
    # distribution info
    total_B_bases = (B_count_input[:B_bins] * B_length_array_bases[:B_bins]).sum()
    B_L1_base_portion = ((B_count_input[0] * len(input_nuc)) / (B_count_input[:B_bins]* B_length_array_bases[:B_bins]).sum()) ### including motif length
    B_nonflank_base_portion = (B_count_input[2:B_bins+2] * B_length_array_bases[:B_bins]).sum() / total_B_bases  ### include portion of triplets 1nt away???
    B_flank_base_portion = (B_count_input[1:B_bins] * 2 * len(input_nuc)).sum() / total_B_bases ### including motif length
    total_A_bases = (A_count_input[:A_bins] * A_length_array_bases[:A_bins]).sum()
    A_nonflank_base_portion = (A_count_input[2:A_bins+2] * A_length_array_bases[:A_bins]).sum() / total_A_bases
    A_flank_base_portion = (A_count_input[1:A_bins] * 2 * len(input_nuc)).sum() / total_A_bases ### including motif length
    total_A_change_in = np.array([0.0]*A_bins); total_B_change_in = np.array([0.0]*B_bins)
    total_A_change_out = np.array([0.0]*A_bins); total_B_change_out = np.array([0.0]*B_bins)
    if mut == True:
        # A>B which adds to the A count locally. add these to A
        A_mut_in_local_A_B = 2 * len(input_nuc) * denovo_sub['Acontraction'] * A_count_input[1:]
        A_mut_out_local_A_B = -2 * len(input_nuc) * denovo_sub['Acontraction'] * A_count_input
        A_mut_out_local_A_B[0] = -1 * len(input_nuc) * A_count_input[0] * denovo_sub['A10']
        # total number of A>B fission events
        A_mut_out_fission = np.insert((-denovo_sub['Afission'] * A_count_input[2:] * A_length_array_bases[:A_bins]), 0, [0, 0]) # used to subtract from A_count, starting from L=3 (with 0 for L=1,2)
        # each fission creates 2 As. add these to A
        A_mut_in_fission = ((2/A_length_array[:A_bins]) * -A_mut_out_fission[2:A_bins+2])[::-1].cumsum()[::-1]
         # B>A which adds to the A count locally (which must come from B_L>1)
        # A from B>A leaving the -1 bin
        A_len_freq = (A_count_input / A_count_input.sum())[:A_bins]
        A_mut_out_local_B_A = -denovo_sub['Aexpansion'] * B_flank_base_portion * total_B_bases * A_len_freq
        # B>A creating A_L=1 from B_L>2
        B_A_into_L1 = total_B_bases * B_nonflank_base_portion * denovo_sub['A01']
        A_mut_in_local_B_A = np.insert(-A_mut_out_local_B_A, 0, B_A_into_L1)     
        # fusion process for A
        A_len_freq = (A_count_input / A_count_input.sum())[:A_bins]
        A_fusion_freq_in = np.bincount((np.add.outer(A_length_array[:A_bins], A_length_array[:A_bins])+1).ravel(), weights = np.outer(A_len_freq, A_len_freq).ravel())[1:]
        A_mut_in_fusion_A_B = A_fusion_freq_in * denovo_sub['Afusion'] * B_L1_base_portion * total_B_bases
        A_mut_out_fusion_A_B = (-2) *A_len_freq * denovo_sub['Afusion'] * B_L1_base_portion * total_B_bases 
        # total B>A
        # B>A which adds to the B count locally. add these to B
        B_mut_in_local_B_A = 2 * len(input_nuc) * denovo_sub['Aexpansion'] * B_count_input[1:]
        B_mut_out_local_B_A = -2 * len(input_nuc) * denovo_sub['Aexpansion'] * B_count_input
        B_mut_out_local_B_A[0] = -1 * B_L1_base_portion * total_B_bases * denovo_sub['Afusion']
        # total number of B>A fission events
        B_mut_out_fission = np.insert((-denovo_sub['A01'] * B_count_input[2:] * B_length_array_bases[:B_bins]), 0, [0, 0]) # used to subtract from B_count, starting from L=3 (with 0 for L=1,2)
        # each fission creates 2 Bs. add these to B
        B_mut_in_fission = ((2/B_length_array[:B_bins]) * -B_mut_out_fission[2:B_bins+2])[::-1].cumsum()[::-1]
        # A>B which adds to the B count locally (which must come from A_L>1)
        # B from A>B leaving the -1 bin
        B_len_freq = (B_count_input / B_count_input.sum())[:B_bins]
        B_mut_out_local_A_B = -denovo_sub['Acontraction'] * A_flank_base_portion * total_A_bases * B_len_freq
        # A>B creating B_L=1 from A_L>2
        A_B_into_L1 = total_A_bases * A_nonflank_base_portion * denovo_sub['Afission']
        B_mut_in_local_A_B = np.insert(-B_mut_out_local_A_B, 0, A_B_into_L1)
        # fusion process for B
        B_len_freq = (B_count_input / B_count_input.sum())[:B_bins]
        B_fusion_freq_in = np.bincount((np.add.outer(B_length_array[:B_bins], B_length_array[:B_bins])+1).ravel(), weights = np.outer(B_len_freq, B_len_freq).ravel())[1:]
        B_mut_in_fusion_B_A = B_fusion_freq_in * denovo_sub['A10'] * A_count_input[0] * len(input_nuc)
        B_mut_out_fusion_B_A = (-2) * B_len_freq * denovo_sub['A10'] * A_count_input[0] * len(input_nuc)
        # update counts for next round (with absorbing boundary)
        total_A_change_in += A_mut_in_local_A_B[:A_bins] + A_mut_in_local_B_A[:A_bins] + A_mut_in_fission[:A_bins] + A_mut_in_fusion_A_B[:A_bins]
        total_B_change_in += B_mut_in_local_B_A[:B_bins] + B_mut_in_local_A_B[:B_bins] + B_mut_in_fission[:B_bins] + B_mut_in_fusion_B_A[:B_bins]
        total_A_change_out += A_mut_out_local_A_B[:A_bins] + A_mut_out_local_B_A[:A_bins] + A_mut_out_fission[:A_bins] + A_mut_out_fusion_A_B[:A_bins]
        total_B_change_out += B_mut_out_local_B_A[:B_bins] + B_mut_out_local_A_B[:B_bins] + B_mut_out_fission[:B_bins] + B_mut_out_fusion_B_A[:B_bins]
        # apply reflecting boundary
        if reflective == True:
            total_A_change_in[A_bins-1] += A_mut_in_local_A_B[A_bins:].sum() + A_mut_in_local_B_A[A_bins:].sum() + A_mut_in_fission[A_bins:].sum() + A_mut_in_fusion_A_B[A_bins:].sum()
            total_B_change_in[B_bins-1] += B_mut_in_local_B_A[B_bins:].sum() + B_mut_in_local_A_B[B_bins:].sum() + B_mut_in_fission[B_bins:].sum() + B_mut_in_fusion_B_A[B_bins:].sum()
            total_A_change_out[A_bins-1] += A_mut_out_local_A_B[A_bins:].sum() + A_mut_out_local_B_A[A_bins:].sum() + A_mut_out_fission[A_bins:].sum() + A_mut_out_fusion_A_B[A_bins:].sum()
            total_B_change_out[B_bins-1] += B_mut_out_local_B_A[B_bins:].sum() + B_mut_out_local_A_B[B_bins:].sum() + B_mut_out_fission[B_bins:].sum() + B_mut_out_fusion_B_A[B_bins:].sum()
           
    if mutonly == False:
        # A expansions in and out
        A_exp_out = A_count_input[:A_bins] * -exp_rate_A_AA[:A_bins]
        A_exp_in = np.insert(-A_exp_out, 0, B_indel_rates[0]*total_B_bases)
        # A contractions in and out
        A_con_out = A_count_input[:A_bins+1] * -con_rate_A_AA[:A_bins+2]
        A_con_in = -A_con_out[1:]
        # A fusions from B1>B0 deletions
        if (mut != True):
            A_len_freq = (A_count_input / A_count_input.sum())[:A_bins]
            A_fusion_freq_in = np.bincount((np.add.outer(A_length_array[:A_bins], A_length_array[:A_bins])+1).ravel(), weights = np.outer(A_len_freq, A_len_freq).ravel())[1:]
        A_mut_in_fusion_Bdel = A_fusion_freq_in[1:A_bins+1] * B_indel_rates[1] * B_L1_base_portion * total_B_bases
        A_mut_out_fusion_Bdel = (-2) *A_len_freq * B_indel_rates[1] * B_L1_base_portion * total_B_bases
        # A fission events from insertions
        A_nonexp_out_fission = -A_count_input * nonexp_rate_A_AB # used to calculate fission_in, starting with L=2 going to 2x L=1
        # each fission creates 2 As. add these to A
        A_nonexp_in_fission = ((2/A_length_array[:A_bins]) * -A_nonexp_out_fission[1:A_bins+1])[::-1].cumsum()[::-1]
        # B expansions in and out
        B_exp_out = B_count_input[:B_bins] * -B_indel_rates[2] * B_length_array[:B_bins] # B>BB rates are flat, per base
        B_exp_in = np.insert(-B_exp_out, 0, A_nonexp_out_fission.sum())
        # B contractions in and out
        B_con_out = B_count_input[:B_bins+1] * -B_indel_rates[1] * B_length_array[:B_bins+1] # B>_ rates are flat, per base
        B_con_in = -B_con_out[1:]
        # B fusions from A1>A0 deletions
        if (mut != True):
            B_len_freq = (B_count_input / B_count_input.sum())[:B_bins]
            B_fusion_freq_in = np.bincount((np.add.outer(B_length_array[:B_bins], B_length_array[:B_bins])+1).ravel(), weights = np.outer(B_len_freq, B_len_freq).ravel())[1:]
        B_mut_in_fusion_Adel = B_fusion_freq_in[1:B_bins+1] * A_count_input[0] * con_rate_A_AA[0]
        B_mut_out_fusion_Adel = 2 *B_len_freq * A_count_input[0] * -con_rate_A_AA[0]
        # B fission events from insertions
        B_nonexp_out_fission = -B_count_input * B_indel_rates[0] * B_length_array # used to calculate fission_in, starting with L=2 going to 2x L=1
        # each fission creates 2 Bs. add these to B
        B_nonexp_in_fission = ((2/B_length_array[:B_bins]) * -B_nonexp_out_fission[1:B_bins+1])[::-1].cumsum()[::-1]
       # update counts for next round (with absorbing boundary)
        total_A_change_in += A_exp_in[:A_bins] + A_con_in[:A_bins] + A_mut_in_fusion_Bdel[:A_bins] + A_nonexp_in_fission[:A_bins]
        total_B_change_in += B_exp_in[:B_bins] + B_con_in[:B_bins] + B_mut_in_fusion_Adel[:B_bins] + B_nonexp_in_fission[:B_bins]
        total_A_change_out += A_exp_out[:A_bins] + A_con_out[:A_bins] + A_mut_out_fusion_Bdel[:A_bins] + A_nonexp_out_fission[:A_bins]
        total_B_change_out += B_exp_out[:B_bins] + B_con_out[:B_bins] + B_mut_out_fusion_Adel[:B_bins] + B_nonexp_out_fission[:B_bins]
        # apply reflecting boundary
        if reflective == True:
            total_A_change_in[A_bins-1] += A_exp_in[A_bins:].sum() + A_con_in[A_bins:].sum() + A_mut_in_fusion_Bdel[A_bins:].sum() + A_nonexp_in_fission[A_bins:].sum()
            total_B_change_in[B_bins-1] += B_exp_in[B_bins:].sum() + B_con_in[B_bins:].sum() + B_mut_in_fusion_Adel[B_bins:].sum() + B_nonexp_in_fission[B_bins:].sum()
            total_A_change_out[A_bins-1] += A_exp_out[A_bins:].sum() + A_con_out[A_bins:].sum() + A_mut_out_fusion_Bdel[A_bins:].sum() + A_nonexp_out_fission[A_bins:].sum()
            total_B_change_out[B_bins-1] += B_exp_out[B_bins:].sum() + B_con_out[B_bins:].sum() + B_mut_out_fusion_Adel[B_bins:].sum() + B_nonexp_out_fission[B_bins:].sum()
    # flag to stop the simulation if more repeats are removed from a bin than exist in that bin (excluding the last 10 noisy bins)
    flag = ((np.abs(total_A_change_out[:A_bins-10]) * speedup_multiplier > A_count_output[:A_bins-10]).sum()) > 0
    # apply speedup
    total_A_change_in *= speedup_multiplier; total_A_change_out *= speedup_multiplier
    total_B_change_in *= speedup_multiplier; total_B_change_out *= speedup_multiplier
    if stochastics is not None:
        # the sum of poisson random variables is poisson-distributed. not necessary to run n poisson samples
        total_A_change_in = np.random.poisson(total_A_change_in.clip(0))
        total_A_change_out = -1 * np.random.poisson(np.abs(total_A_change_out.clip(max=0)))
        total_B_change_in = np.random.poisson(total_B_change_in.clip(0))
        total_B_change_out = -1 * np.random.poisson(np.abs(total_B_change_out.clip(max=0)))   
    total_A_change = total_A_change_in + total_A_change_out
    total_B_change = total_B_change_in + total_B_change_out
    # update counts for next round
    A_count_output = A_count_output[:A_bins] + total_A_change[:A_bins]
    B_count_output = B_count_output[:B_bins] + total_B_change[:B_bins]
    # remove negative values
    A_count_output[A_count_output <0] = 0            
    B_count_output[B_count_output <0] = 0
    boundary_flag = ((A_count_output[A_bins-1] > boundary_count) | (B_count_output[B_bins-1] > boundary_count))
    if output_components == True:
        if mutonly == False:
            return  A_mut_in_local_A_B[:A_bins], A_mut_out_local_A_B[:A_bins], A_mut_in_local_B_A[:A_bins], A_mut_out_local_B_A[:A_bins], A_mut_in_fission[:A_bins], A_mut_out_fission[:A_bins], A_mut_in_fusion_A_B[:A_bins], A_mut_out_fusion_A_B[:A_bins], A_exp_in[:A_bins], A_exp_out[:A_bins], A_con_in[:A_bins], A_con_out[:A_bins], A_mut_in_fusion_Bdel[:A_bins], A_mut_out_fusion_Bdel[:A_bins], A_nonexp_in_fission[:A_bins], A_nonexp_out_fission[:A_bins]
        else:
            return  A_mut_in_local_A_B[:A_bins], A_mut_out_local_A_B[:A_bins], A_mut_in_local_B_A[:A_bins], A_mut_out_local_B_A[:A_bins], A_mut_in_fission[:A_bins], A_mut_out_fission[:A_bins], A_mut_in_fusion_A_B[:A_bins], A_mut_out_fusion_A_B[:A_bins]
    else:
        return A_count_output, B_count_output, flag, boundary_flag

In [ ]:
def pin_power_law(power, pin_rate, pin_len=9, start_len=1, end_len=200):
    denom = (pin_len**power) / pin_rate
    return pd.Series([i**power for i in range(start_len, end_len+1)], index = list(range(start_len,end_len+1))) / denom

def intercept_then_powerlaw(exp_power, con_power, exp_int, con_int, pin_len = 9, A_bins = 200, boot = None, motif = 'A', interp = False, nonexp_factor = 0.01, intercept_list = intercept_list):
    if boot is None:
        bootname = ''
        denovo_exp_rate_current = denovo_exp_rate[motif].copy()
        denovo_con_rate_current = denovo_con_rate[motif].copy()
        denovo_nonexp_rate_current = denovo_nonexp_rate[motif].copy()
    else:
        bootname = '_boot'+str(boot)
        denovo_exp_rate_current = denovo_exp_rate_poisson[motif][boot].copy()
        denovo_con_rate_current = denovo_con_rate_poisson[motif][boot].copy()
        denovo_nonexp_rate_current = denovo_nonexp_rate_poisson[motif][boot].copy()
    exp = pd.concat([denovo_exp_rate_current.reindex(range(pin_len)), pin_power_law(exp_power, intercept_list[exp_int], start_len=pin_len, end_len=A_bins+3)])
    con = pd.concat([denovo_con_rate_current.reindex(range(pin_len)), pin_power_law(con_power, intercept_list[con_int], start_len=pin_len, end_len=A_bins+3)])
    nonexp = pd.concat([denovo_nonexp_rate_current.reindex(range(pin_len)), pin_power_law(exp_power, intercept_list[exp_int] * nonexp_factor, start_len=pin_len, end_len=A_bins+3)])
    nonexpname = '_nex' + str(nonexp_factor)
    nonexp.loc[1] = 0
    if interp == True:
        interpname = '_interp'
        exp.loc[8:13] = np.nan
        con.loc[8:13] = np.nan
        nonexp.loc[8:13] = np.nan
        exp = exp.interpolate(method = 'quadratic')
        con = con.interpolate(method = 'quadratic')
        nonexp = nonexp.interpolate(method = 'quadratic')
    else:
        interpname = ''
    name = '_interceptPL_tE' + str(exp_power) + '_tC' + str(con_power) +'_iE' + str(exp_int) + '_iC' + str(con_int) + nonexpname + interpname + bootname
    return name, exp, con, nonexp

#### alternate rate functional forms
def power_law_rate_at_L(power, L, rate = 1e-8, bins = 200):
    return pd.Series([np.exp(np.log(rate) - power * np.log(L) + np.log(length)*power) for length in range(1,bins)], index = range(1,bins))

def log_curve_from_L9(power, intercept, bins = 200):
    return pd.Series([intercept_list[intercept] * (np.log(1+L-8) / np.log(1+9 -8))**power for L in range(9,bins+1)], index = range(9,bins+1))

In [ ]:
# setup function
def setup_evolve(exp_int, con_int, exp_power, con_power, starting_counts, boot = None, stochastics = None, interp = False, pin_len = 9, nonexp_factor = 0.01, A_bins = 200, B_bins = 200, input_nuc = 'A', exp_zero = False, con_zero = False, nonexp_zero = False, ceiling = None, rates_function = 'powerlaw'):
# set up counts
    A_length_array = np.array(range(1,A_bins+1))
    B_length_array = np.array(range(1,A_bins+1))

    A_count_input = np.nan_to_num(starting_counts[0].reindex(range(1,A_bins+1)).values)
    B_count_input = np.nan_to_num(starting_counts[1].reindex(range(1,B_bins+1)).values)
# set up rates    
    if rates_function == 'powerlaw':
        name, exp_rate, con_rate, nonexp_rate = intercept_then_powerlaw(exp_power = exp_power, con_power = con_power, exp_int = exp_int, con_int=con_int, pin_len=pin_len, A_bins = A_bins, boot = boot, motif = input_nuc, interp = interp, nonexp_factor = nonexp_factor)
        B_indel_rate = np.array([exp_rate[0], con_rate[0], nonexp_rate[0]])        
    if rates_function == 'powerlaw_x':
        name = '_PL1e-8atL_tE' + str(exp_power) + '_tC' + str(con_power) +'_LE' + str(exp_int) + '_LC' + str(con_int)
        exp_rate = power_law_rate_at_L(exp_power, exp_int, bins = A_bins+3).reindex(range(A_bins+3))
        con_rate = power_law_rate_at_L(con_power, con_int, bins = A_bins+3).reindex(range(A_bins+3))
        nonexp_rate = exp_rate * nonexp_factor
        B_indel_rate = np.array([denovo_exp_rate[input_nuc][0], denovo_con_rate[input_nuc][0], denovo_nonexp_rate[input_nuc][0]])
    if rates_function == 'log':
        name = '_logrates_tE' + str(exp_power) + '_tC' + str(con_power) +'_LE' + str(exp_int) + '_LC' + str(con_int)
        exp_rate = pd.concat([denovo_exp_rate['A'].loc[:8], log_curve_from_L9(exp_power, exp_int, A_bins+3)])
        con_rate = pd.concat([denovo_con_rate['A'].loc[:8], log_curve_from_L9(con_power, con_int, A_bins+3)])
        nonexp_rate = pd.concat([denovo_nonexp_rate['A'].loc[:8], log_curve_from_L9(exp_power, exp_int, A_bins+3) * nonexp_factor])
        B_indel_rate = np.array([denovo_exp_rate[input_nuc][0], denovo_con_rate[input_nuc][0], denovo_nonexp_rate[input_nuc][0]])
    if rates_function == 'custom':
        exp_rate, con_rate, name = custom_rates
        exp_rate = exp_rate.reindex(range(A_bins+3))
        con_rate = con_rate.reindex(range(A_bins+3))
        nonexp_rate = exp_rate * nonexp_factor
        B_indel_rate = np.array([denovo_exp_rate[input_nuc][0], denovo_con_rate[input_nuc][0], denovo_nonexp_rate[input_nuc][0]])
    # change rates from per unit to per STR
    exp_rate = exp_rate.values[1:A_bins+1] * A_length_array
    con_rate = con_rate.values[1:A_bins+2] * np.array(range(1,A_bins+2))
    nonexp_rate = nonexp_rate.values[1:A_bins+3] * np.array(range(1,A_bins+3))
    if ceiling != None:
        ceiling_loc = []
        if (exp_rate > ceiling).sum() > 0:
            ceiling_loc.append(pd.Series(exp_rate > ceiling).idxmax())
        if (con_rate > ceiling).sum() > 0:
            ceiling_loc.append(pd.Series(con_rate > ceiling).idxmax())
        if len(ceiling_loc) > 0:
            ceiling_loc = min(np.array(ceiling_loc))
            print( '\r' + 'rate ceiling reached at L=' + str(ceiling_loc), end = ' ')
            exp_rate[ceiling_loc:] = ceiling
            con_rate[ceiling_loc:] = ceiling
            nonexp_rate[ceiling_loc:] = ceiling
        name = name + '_ceiling_' + str(ceiling)
    return A_count_input, B_count_input, exp_rate, con_rate, nonexp_rate, B_indel_rate, name

In [ ]:
def plot_relative_processes(exp_int, con_int, exp_power, con_power, starting_counts, interp = False, pin_len = 9, nonexp_factor = 0.01, motif = 'A', ceiling = None, rates_function = 'powerlaw', show = False, norm = True, returndata = False, separate = False, A_bins = 200, B_bins = 200, cut_at_n1 = False):
    starting_conditions = setup_evolve(exp_int, con_int, exp_power, con_power, starting_counts = starting_counts, pin_len = pin_len, interp = interp, nonexp_factor = nonexp_factor, ceiling = ceiling, rates_function = rates_function, input_nuc = motif, A_bins = A_bins, B_bins = B_bins)
    A_components = mut_evolve_dist_AB(starting_conditions[0][:A_bins], starting_conditions[1][:B_bins], starting_conditions[2:6], output_components=True, input_nuc = motif, speedup_multiplier = 0)
    A_components = pd.DataFrame(A_components)
    A_components.columns = A_components.columns + 1
    A_components.index = ['mut_in_local_A_B', 'mut_out_local_A_B', 'mut_in_local_B_A', 'mut_out_local_B_A', 'mut_in_fission', 'mut_out_fission', 'mut_in_fusion_A_B', 'mut_out_fusion_A_B', 'exp_in', 'exp_out', 'con_in', 'con_out', 'mut_in_fusion_Bdel', 'mut_out_fusion_Bdel', 'nonexp_in_fission', 'nonexp_fissions_out']
    A_components_in = A_components.loc[A_components.index[::2]]
    A_components_out = A_components.loc[A_components.index[1::2]]
    A_components_in.index = ['substitution -1', 'substitution +1', 'substitution fission', 'substitution fusion', 'expansion +1', 'contraction -1', 'deletion fusion', 'non-motif insertion fission']
    A_components_out.index = ['substitution -1', 'substitution +1', 'substitution fission', 'substitution fusion', 'expansion +1', 'contraction -1', 'deletion fusion', 'non-motif insertion fission']
    A_components_sum = A_components_in + A_components_out
    if norm == True:
        abs_total = A_components_in.abs().sum() + A_components_out.abs().sum()
        A_components_in = A_components_in / abs_total
        A_components_out = A_components_out / abs_total
        A_components_sum = A_components_sum / A_components_sum.abs().sum()
    if returndata == 'sum':
        return A_components_sum
    if returndata == 'in':
        return A_components_in
    if returndata == 'out':
        return A_components_out
    else:
        last_int_bin = (pd.Series(starting_conditions[0]) >= 1).idxmin()
        fig_components = go.Figure()
        if separate == True:
            for component in A_components_sum.index:
                fig_components.add_trace(go.Bar(name = component, x = A_components_in.columns, y = A_components_in.loc[component], legendgroup = component))
            for component in A_components_sum.index:
                fig_components.add_trace(go.Bar(name = component, x = A_components_out.columns, y = A_components_out.loc[component], legendgroup = component, showlegend = False))
        else:
            for component in A_components_sum.index:
                fig_components.add_trace(go.Bar(name = component, x = A_components_sum.columns, y = A_components_sum.loc[component]))
        fig_components.add_trace(go.Scatter(x = [last_int_bin] *2, y = [-0.505, 0.55], name = 'count <1', mode = 'lines', line = dict(color = 'rgba(0,0,0,0.5)', dash = 'dash', width = 3)))
        fig_components.update_layout(barmode='relative', colorway = plotly.colors.qualitative.Plotly[:8], height = 250, margin={'t':20,'l':60,'b':40,'r':10})        
        if cut_at_n1 == True:
            if last_int_bin > 10:
                fig_components.update_xaxes(range = [0.5, last_int_bin + 0.5], title = 'repeat length (nt)')
                fig_components.update_yaxes(range = [-0.51, 0.51])
        if cut_at_n1 == 'sup':
            if last_int_bin > 10:
                fig_components.update_xaxes(range = [0.5, last_int_bin + 0.5], title = 'repeat length (nt)')
                fig_components.update_yaxes(range = [-0.51, 0.51])
        else:
            fig_components.update_xaxes(range = [0.5, A_bins + 0.5], title = 'repeat length (nt)')
        fig_components.update_yaxes(title = 'relative contribution', gridcolor = 'rgba(0,0,0,0.5)')
        if show == True:
            fig_components.show()
        return fig_components, last_int_bin

In [ ]:
def plot_flux_combinations(comb_list, separate = False, show_plot = True, showlegend = True, showlabels = True, write = False, cut_at_n1 = False, norm = True, legend_size = False, A_bins = 200, xpos = None):
    all_plots = dict()
    if xpos is None:
        xpos = pd.Series([0] * len(comb_list), index = [parameters for parameters in comb_list])
    else:
        xpos = pd.Series(xpos, index = [tuple(parameters) for parameters in comb_list])
    for parameters in comb_list:
        fig_flux, last_int_bin = plot_relative_processes(parameters[0],parameters[1],parameters[2],parameters[3], [inference_grid_final_nointerp_subonly[parameters[0]][parameters[1]][parameters[2]][parameters[3]], inference_grid_final_nointerp_subonly_Bdist[parameters[0]][parameters[1]][parameters[2]][parameters[3]]], norm=norm, separate=separate, cut_at_n1=cut_at_n1, A_bins=A_bins)
        if cut_at_n1 == False:
            fig_flux.update_xaxes(range = [0.5, 99])
        fig_flux.update_xaxes(dtick = 10)
        fig_flux.update_layout(font=dict(family = 'Arial', size = 12), title = str((parameters[2],parameters[3])), height = 210, width = 550, margin={'t':60,'l':60,'b':40,'r':10}, showlegend = showlegend)
        if legend_size == True:
            fig_flux.update_layout(font=dict(family = 'Arial', size = 12), title = str((parameters[2],parameters[3])), height = 500, width = 500, margin={'t':60,'l':60,'b':40,'r':10}, showlegend = showlegend)
        if cut_at_n1 == True:
            if last_int_bin > 10:
                fig_flux.update_xaxes(range = [0.5, last_int_bin+0.5])

                fig_flux.update_layout(width = 80 + 7*last_int_bin)
            else:
                fig_flux.update_xaxes(dtick = 50)
        if showlabels == False:
            fig_flux.update_xaxes(title = None, position = xpos[tuple(parameters)]); fig_flux.update_yaxes(title = None); fig_flux.update_layout(title = None, height = 140, margin={'t':10,'l':60,'b':20,'r':10})
        if show_plot == True:
            fig_flux.show()
        if write != False:
            if separate == False:
                if cut_at_n1 == 'sup':
                    fig_flux.write_image('plots/fig_sup_flux_'+str(parameters[0])+'_'+str(parameters[1])+'_'+str(parameters[2])+'_'+str(parameters[3])+'.'+ write)
                else:
                    fig_flux.write_image('plots/fig_flux_'+str(parameters[0])+'_'+str(parameters[1])+'_'+str(parameters[2])+'_'+str(parameters[3])+'.'+ write)
            else:
                if cut_at_n1 == 'sup':
                    fig_flux.write_image('plots/fig_sup_flux_separated_'+str(parameters[0])+'_'+str(parameters[1])+'_'+str(parameters[2])+'_'+str(parameters[3])+'.'+ write)
                else:
                    fig_flux.write_image('plots/fig_flux_separated_'+str(parameters[0])+'_'+str(parameters[1])+'_'+str(parameters[2])+'_'+str(parameters[3])+'.'+ write)
        all_plots[str(list(parameters))] = fig_flux
    return all_plots

#### Plots for Fig. 6c

In [ ]:
parameter_combination_examples = [[4,3,0,3.5], [4,3,0.5, 3], [4,3,1,2.5], [4,3,1.1,2.4], [4,3,1.2,2.3],  [4,3,1.3,2.2],  [4,3,1.4,2.1], [4,3, 1.5, 2], [4,3, 1.6, 1.9], [4,3, 2.8, 3.3], [4,3, 1.7, 1.8], [4,3, 1.7, 1.7], [4,3, 2.5, 1], [3,2,0,0]]
flux_plots = plot_flux_combinations(parameter_combination_examples, showlegend = False, separate = False, cut_at_n1=True, A_bins=200, show_plot=False, showlabels = False, xpos = [0,0,0,0,0,0,0,0,0,0,0, 0.05, 0.05, 0], write = False)#'pdf')

In [ ]:
# Show example
flux_plots['[4, 3, 1.5, 2]'].show()

#### additional plots for Supplemental Note 1

In [ ]:
flux_plots_a = plot_flux_combinations(parameter_combination_examples, showlegend = False, separate = False, cut_at_n1='sup', A_bins=200, show_plot=False, showlabels = False, xpos = [0,0,0,0,0,0,0,0,0,0,0, 0.05, 0.05, 0], write = 'pdf')

In [ ]:
flux_plots_b = plot_flux_combinations(parameter_combination_examples, showlegend = False, separate = True, cut_at_n1='sup', A_bins=200, show_plot=False, showlabels = False, xpos = [0,0,0,0,0,0,0,0,0,0,0, 0.05, 0.05, 0], write = 'pdf')

In [ ]:
def plot_net_flux_lineplot(comb_list, show_sum = False, show_plot = True, show_legend = True, showlabels = True, write = False, A_bins = 200, xpos = None):
    all_plots = dict()
    for parameters in comb_list:      
        data = plot_relative_processes(parameters[0],parameters[1],parameters[2],parameters[3], [inference_grid_final_nointerp_subonly[parameters[0]][parameters[1]][parameters[2]][parameters[3]], inference_grid_final_nointerp_subonly_Bdist[parameters[0]][parameters[1]][parameters[2]][parameters[3]]], A_bins=200, norm = False, returndata = 'sum')
        data_ins = plot_relative_processes(parameters[0],parameters[1],parameters[2],parameters[3], [inference_grid_final_nointerp_subonly[parameters[0]][parameters[1]][parameters[2]][parameters[3]], inference_grid_final_nointerp_subonly_Bdist[parameters[0]][parameters[1]][parameters[2]][parameters[3]]], A_bins=200, norm = False, returndata = 'in')
        data_outs = plot_relative_processes(parameters[0],parameters[1],parameters[2],parameters[3], [inference_grid_final_nointerp_subonly[parameters[0]][parameters[1]][parameters[2]][parameters[3]], inference_grid_final_nointerp_subonly_Bdist[parameters[0]][parameters[1]][parameters[2]][parameters[3]]], A_bins=200, norm = False, returndata = 'out')
        Lmax = (inference_grid_final_nointerp_subonly[parameters[0]][parameters[1]][parameters[2]][parameters[3]] >= 1).idxmin()
        if Lmax < 10:
            Lmax = 200
        components_fission_in = data_ins.loc['substitution fission'] + data_ins.loc['non-motif insertion fission']
        components_fission_out = data_outs.loc['substitution fission'] + data_outs.loc['non-motif insertion fission']
        local_terms = data_ins.loc['expansion +1'] + data_ins.loc['contraction -1'] + (data_outs.loc['expansion +1'] + data_outs.loc['contraction -1'])
        local_terms_wsub = data_ins.loc['expansion +1'] + data_ins.loc['contraction -1'] + data_ins.loc['substitution +1'] + data_ins.loc['substitution -1'] + (data_outs.loc['expansion +1'] + data_outs.loc['contraction -1'] + data_outs.loc['substitution +1'] + data_outs.loc['substitution -1'])
        local_subonly = data_ins.loc['substitution +1'] + data_ins.loc['substitution -1'] + (data_outs.loc['substitution +1'] + data_outs.loc['substitution -1'])
        all_subonly = data_ins.loc['substitution +1'] + data_ins.loc['substitution -1'] + data_ins.loc['substitution fission'] + data_ins.loc['substitution fusion'] + (data_outs.loc['substitution +1'] + data_outs.loc['substitution -1'] + data_outs.loc['substitution fission'] + data_outs.loc['substitution fusion'])
        local_plus_fission_out = local_terms + components_fission_out
        local_plus_fission_all = local_terms + components_fission_in + components_fission_out
        all_terms = (data_ins + data_outs).sum()
       
        fig_components = go.Figure()
        fig_components.add_trace(go.Scatter(x = all_terms.index[:Lmax], y = all_terms[:Lmax], mode = 'lines', line = dict(color = 'rgba(1,1,1,0.8)', width = 3), name = 'computational model: all terms'))
        fig_components.add_trace(go.Scatter(x = local_terms.index[:Lmax], y = local_terms[:Lmax], mode = 'lines', line = dict(color = 'rgba(255,178,101,0.8)', width = 3), name = 'local terms (no fission, no fusion)'))
        fig_components.add_trace(go.Scatter(x = local_plus_fission_out.index[:Lmax], y = local_plus_fission_out[:Lmax], mode = 'lines', line = dict(color = 'rgba(255,101,101,0.8)', width = 3), name = 'no fission in, no fusion'))
        fig_components.add_trace(go.Scatter(x = local_plus_fission_all.index[:Lmax], y = local_plus_fission_all[:Lmax], mode = 'lines', line = dict(color = 'rgba(178,101,178,0.8)', width = 3), name = 'no fusion'))
        fig_components.update_xaxes(title = 'repeat length (nt)')
        yrange = max(0.05, all_terms[5:15].abs().max())
        fig_components.update_yaxes(title = 'net flux (mut. per gen.)', range = [-yrange, yrange])
        fig_components.update_layout(font=dict(family = 'Arial', size = 12), title = str((float(round(m_list[parameters[1]],1)),parameters[2],parameters[3])), height = 210, width = 550, margin={'t':60,'l':60,'b':40,'r':10}, showlegend = show_legend)
        if show_plot == True:
            fig_components.show()
        if show_sum == True:
            print(all_terms.sum())
        if write != False:
            fig_components.write_image('plots/fig_netfluxlines_finaltimepoint_'+str(round(m_list[parameters[1]],1))+'_'+str(parameters[2])+'_'+str(parameters[3])+'.'+ write)
        all_plots[str(list(parameters))] = fig_components
    return all_plots

In [ ]:
flux_plots_c = plot_net_flux_lineplot(parameter_combination_examples, show_plot=False, show_legend=False, write = 'pdf')

In [ ]:
# Show examples
flux_plots_a['[4, 3, 1.5, 2]'].show()

In [ ]:
flux_plots_b['[4, 3, 1.5, 2]'].show()

In [ ]:
flux_plots_c['[4, 3, 1.5, 2]'].show()

## Substitution-only simulations
- data processed in Notebook 3

In [ ]:
subonly_counts_unit = subonly_counts['A'].T.reset_index().dropna(how='all', axis=1)
subonly_counts_unit['unit'] = subonly_counts_unit['index'].str.len()
subonly_counts_unit = subonly_counts_unit.set_index(['unit', 'index']).groupby(['unit']).sum().transpose()
CHM13_counts_byunit = CHM13_counts.T.groupby(['unit_len']).sum().transpose()

In [ ]:
# Fig. S15b
fig_mutonly = go.Figure()
for unit in [1,2,3]:
    fig_mutonly.add_trace(go.Scatter(x = CHM13_counts_byunit.index * unit, y = CHM13_counts_byunit[unit] / CHM13_counts_byunit[unit].sum(), opacity = 0.6, mode = 'lines', line = dict(width = 3), legendgroup = unit, name = 'unit='+str(unit)))
for unit in [1,2,3]:
    fig_mutonly.add_trace(go.Scatter(x = subonly_counts_unit.index * unit, y = subonly_counts_unit[unit] / subonly_counts_unit[unit].sum(), opacity = 0.5, mode = 'lines', line = dict(dash = 'dash', width = 3), legendgroup = unit, name = 'substitutions only'))

fig_mutonly.update_yaxes(type = 'log', title = 'repeat counts (normalized)', range = [-9.05,0], tickformat = '1.0e', dtick = 2, gridcolor = 'rgba(0,0,0,0.2)')
fig_mutonly.update_xaxes(type = 'log', title = 'repeat tract length (nt)', range = [0,2.2], tickvals = [1,2,5,10,20,50,100], gridcolor = 'rgba(0,0,0,0.2)')
fig_mutonly.update_layout(font=dict(family = 'Arial', size = 14), height = 400, width = 520, margin={'t':20,'l':65,'b':45,'r':20}, colorway = plotly.colors.DEFAULT_PLOTLY_COLORS[:3], legend=dict(yanchor="top", y=0.99, xanchor="right", x=1.0))

In [ ]:
fig_mutonly.write_image('plots/figS15b.pdf')